In [1]:
# find [why] for questions
# why I get error with eval_type = read_int(infile)

import pyspark
from pyspark.sql import SparkSession, Window
from pyspark.sql.functions import *
from pyspark.sql.types import *

spark = SparkSession.builder.appName("Functions 3.5.0").getOrCreate()
spark

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/10/22 15:48:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
path = "datasets/"
students = spark.read.csv(path+"students.csv", inferSchema=True, header=True)
tour = spark.read.csv(path+"pga_tour_historical.csv", inferSchema=True, header=True)
tour.limit(100).dropna().dropDuplicates().limit(5).show()
students.limit(5).toPandas()

+---------------+------+----------------+--------------------+-----+
|    Player Name|Season|       Statistic|            Variable|Value|
+---------------+------+----------------+--------------------+-----+
|Robert Garrigus|  2010|Driving Distance|Driving Distance ...|   71|
|   Bubba Watson|  2010|Driving Distance|Driving Distance ...|   77|
| Dustin Johnson|  2010|Driving Distance|Driving Distance ...|   83|
|Brett Wetterich|  2010|Driving Distance|Driving Distance ...|   54|
|    J.B. Holmes|  2010|Driving Distance|Driving Distance ...|  100|
+---------------+------+----------------+--------------------+-----+



gender race/ethnicity parental level of education         lunch  \
0  female        group B           bachelor's degree      standard   
1  female        group C                some college      standard   
2  female        group B             master's degree      standard   
3    male        group A          associate's degree  free/reduced   
4    male        group C                some college      standard   

  test preparation course  math score  reading score  writing score  
0                    none          72             72             74  
1               completed          69             90             88  
2                    none          90             95             93  
3                    none          47             57             44  
4                    none          76             78             75

# Functions

## Normal Functions¶

## Math Functions

## Datetime Functions

## Collection Functions

## Partition Transformation Functions

## Aggregate Functions

## [Window Functions](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/functions.html#window-functions)

#### pyspark.sql.functions.cume_dist() → pyspark.sql.column.Column
Window function: returns the cumulative distribution of values within a window partition, i.e. the fraction of rows that are below the current row.

In [201]:
df = spark.createDataFrame([1, 2, 3, 3, 4], IntegerType())
w = Window.orderBy("value")
df.withColumn("cd", cume_dist().over(w)).show()

23/10/22 20:48:12 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/10/22 20:48:12 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/10/22 20:48:12 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+-----+---+
|value| cd|
+-----+---+
|    1|0.2|
|    2|0.4|
|    3|0.8|
|    3|0.8|
|    4|1.0|
+-----+---+



23/10/22 20:48:12 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/10/22 20:48:12 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


In [205]:
w = Window.orderBy("math score")
students.orderBy(desc("reading score")).limit(50).withColumn("cd", cume_dist().over(w)).show(50)

+------+--------------+---------------------------+------------+-----------------------+----------+-------------+-------------+----+
|gender|race/ethnicity|parental level of education|       lunch|test preparation course|math score|reading score|writing score|  cd|
+------+--------------+---------------------------+------------+-----------------------+----------+-------------+-------------+----+
|female|       group B|         associate's degree|free/reduced|              completed|        76|           94|           87|0.02|
|female|       group B|            master's degree|free/reduced|              completed|        77|           97|           94|0.06|
|female|       group C|          bachelor's degree|    standard|              completed|        77|           94|           95|0.06|
|female|       group D|           some high school|    standard|                   none|        81|           97|           96|0.08|
|female|       group D|               some college|    standard|     

23/10/22 20:50:40 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/10/22 20:50:40 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/10/22 20:50:40 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


#### pyspark.sql.functions.dense_rank() → pyspark.sql.column.Column
Window function: returns the rank of rows within a window partition, without any gaps.

The difference between rank and dense_rank is that dense_rank leaves no gaps in ranking sequence when there are ties. That is, if you were ranking a competition using dense_rank and had three people tie for second place, you would say that all three were in second place and that the next person came in third. Rank would give me sequential numbers, making the person that came in third place (after the ties) would register as coming in fifth.

This is equivalent to the DENSE_RANK function in SQL.

In [206]:
df = spark.createDataFrame([1, 1, 2, 3, 3, 4], types.IntegerType())
w = Window.orderBy("value")
df.withColumn("drank", dense_rank().over(w)).show()

23/10/22 20:51:57 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/10/22 20:51:57 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/10/22 20:51:57 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+-----+-----+
|value|drank|
+-----+-----+
|    1|    1|
|    1|    1|
|    2|    2|
|    3|    3|
|    3|    3|
|    4|    4|
+-----+-----+



In [207]:
w = Window.orderBy("math score")
students.orderBy(desc("reading score")).limit(50).withColumn("drank", dense_rank().over(w)).show(50)

+------+--------------+---------------------------+------------+-----------------------+----------+-------------+-------------+-----+
|gender|race/ethnicity|parental level of education|       lunch|test preparation course|math score|reading score|writing score|drank|
+------+--------------+---------------------------+------------+-----------------------+----------+-------------+-------------+-----+
|female|       group B|         associate's degree|free/reduced|              completed|        76|           94|           87|    1|
|female|       group B|            master's degree|free/reduced|              completed|        77|           97|           94|    2|
|female|       group C|          bachelor's degree|    standard|              completed|        77|           94|           95|    2|
|female|       group D|           some high school|    standard|                   none|        81|           97|           96|    3|
|female|       group D|               some college|    standar

23/10/22 20:52:33 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/10/22 20:52:33 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/10/22 20:52:33 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


#### pyspark.sql.functions.lag(col: ColumnOrName, offset: int = 1, default: Optional[Any] = None) → pyspark.sql.column.Column
Window function: returns the value that is offset rows before the current row, and default if there is less than offset rows before the current row. For example, an offset of one will return the previous row at any given point in the window partition.

This is equivalent to the LAG function in SQL.

Parameters:
- col: Column or str | 
name of column or expression
- offset: int, optional default 1 | 
number of row to extend
- default: optional | 
default value

Returns – Column | 
value before current row based on offset.

In [209]:
df = spark.createDataFrame([("a", 1),
                            ("a", 2),
                            ("a", 3),
                            ("b", 8),
                            ("b", 2)], ["c1", "c2"])
df.show()

w = Window.partitionBy("c1").orderBy("c2")
df.withColumn("previos_value", lag("c2").over(w)).show()

+---+---+
| c1| c2|
+---+---+
|  a|  1|
|  a|  2|
|  a|  3|
|  b|  8|
|  b|  2|
+---+---+

+---+---+-------------+
| c1| c2|previos_value|
+---+---+-------------+
|  a|  1|         NULL|
|  a|  2|            1|
|  a|  3|            2|
|  b|  2|         NULL|
|  b|  8|            2|
+---+---+-------------+



In [210]:
df.withColumn("previos_value", lag("c2", 1, 0).over(w)).show()

+---+---+-------------+
| c1| c2|previos_value|
+---+---+-------------+
|  a|  1|            0|
|  a|  2|            1|
|  a|  3|            2|
|  b|  2|            0|
|  b|  8|            2|
+---+---+-------------+



In [211]:
df.withColumn("previos_value", lag("c2", 2, -1).over(w)).show()

+---+---+-------------+
| c1| c2|previos_value|
+---+---+-------------+
|  a|  1|           -1|
|  a|  2|           -1|
|  a|  3|            1|
|  b|  2|           -1|
|  b|  8|           -1|
+---+---+-------------+



#### pyspark.sql.functions.lead(col: ColumnOrName, offset: int = 1, default: Optional[Any] = None) → pyspark.sql.column.Column
Window function: returns the value that is offset rows after the current row, and default if there is less than offset rows after the current row. For example, an offset of one will return the next row at any given point in the window partition.

This is equivalent to the LEAD function in SQL.

Parameters:
- col: Column or str | 
name of column or expression
- offset: int, optional default 1 | 
number of row to extend
- default: optional | 
default value

Returns – Column | 
value after current row based on offset.

In [212]:
df = spark.createDataFrame([("a", 1),
                            ("a", 2),
                            ("a", 3),
                            ("b", 8),
                            ("b", 2)], ["c1", "c2"])
df.show()

+---+---+
| c1| c2|
+---+---+
|  a|  1|
|  a|  2|
|  a|  3|
|  b|  8|
|  b|  2|
+---+---+



In [213]:
w = Window.partitionBy("c1").orderBy("c2")
df.withColumn("next_value", lead("c2").over(w)).show()

+---+---+----------+
| c1| c2|next_value|
+---+---+----------+
|  a|  1|         2|
|  a|  2|         3|
|  a|  3|      NULL|
|  b|  2|         8|
|  b|  8|      NULL|
+---+---+----------+



In [214]:
df.withColumn("next_value", lead("c2", 1, 0).over(w)).show()

+---+---+----------+
| c1| c2|next_value|
+---+---+----------+
|  a|  1|         2|
|  a|  2|         3|
|  a|  3|         0|
|  b|  2|         8|
|  b|  8|         0|
+---+---+----------+



In [215]:
df.withColumn("next_value", lead("c2", 2, -1).over(w)).show()

+---+---+----------+
| c1| c2|next_value|
+---+---+----------+
|  a|  1|         3|
|  a|  2|        -1|
|  a|  3|        -1|
|  b|  2|        -1|
|  b|  8|        -1|
+---+---+----------+



#### pyspark.sql.functions.nth_value(col: ColumnOrName, offset: int, ignoreNulls: Optional[bool] = False) → pyspark.sql.column.Column
Window function: returns the value that is the offsetth row of the window frame (counting from 1), and null if the size of window frame is less than offset rows.

It will return the offsetth non-null value it sees when ignoreNulls is set to true. If all values are null, then null is returned.

This is equivalent to the nth_value function in SQL.

Parameters:
- col: Column or str | 
name of column or expression
- offset: int | 
number of row to use as the value
- ignoreNulls: bool, optional | 
indicates the Nth value should skip null in the determination of which row to use

Returns – Column | 
value of nth row.

In [217]:
from pyspark.sql import Window
df = spark.createDataFrame([("a", 1),
                            ("a", 2),
                            ("a", 3),
                            ("b", 8),
                            ("b", 2)], ["c1", "c2"])
df.show()

+---+---+
| c1| c2|
+---+---+
|  a|  1|
|  a|  2|
|  a|  3|
|  b|  8|
|  b|  2|
+---+---+



In [218]:
w = Window.partitionBy("c1").orderBy("c2")
df.withColumn("nth_value", nth_value("c2", 1).over(w)).show()

+---+---+---------+
| c1| c2|nth_value|
+---+---+---------+
|  a|  1|        1|
|  a|  2|        1|
|  a|  3|        1|
|  b|  2|        2|
|  b|  8|        2|
+---+---+---------+



In [219]:
df.withColumn("nth_value", nth_value("c2", 2).over(w)).show()

+---+---+---------+
| c1| c2|nth_value|
+---+---+---------+
|  a|  1|     NULL|
|  a|  2|        2|
|  a|  3|        2|
|  b|  2|     NULL|
|  b|  8|        8|
+---+---+---------+



#### pyspark.sql.functions.ntile(n: int) → pyspark.sql.column.Column
Window function: returns the ntile group id (from 1 to n inclusive) in an ordered window partition. For example, if n is 4, the first quarter of the rows will get value 1, the second quarter will get 2, the third quarter will get 3, and the last quarter will get 4.

This is equivalent to the NTILE function in SQL.

In [222]:
df = spark.createDataFrame([("a", 1),
                            ("a", 2),
                            ("a", 3),
                            ("b", 8),
                            ("b", 2)], ["c1", "c2"])
df.show()

+---+---+
| c1| c2|
+---+---+
|  a|  1|
|  a|  2|
|  a|  3|
|  b|  8|
|  b|  2|
+---+---+



In [221]:
w = Window.partitionBy("c1").orderBy("c2")
df.withColumn("ntile", ntile(2).over(w)).show()

+---+---+-----+
| c1| c2|ntile|
+---+---+-----+
|  a|  1|    1|
|  a|  2|    1|
|  a|  3|    2|
|  b|  2|    1|
|  b|  8|    2|
+---+---+-----+



In [227]:
w = Window.partitionBy("parental level of education").orderBy("math score")
students.withColumn("ntile", ntile(10).over(w)).show(1000)

+------+--------------+---------------------------+------------+-----------------------+----------+-------------+-------------+-----+
|gender|race/ethnicity|parental level of education|       lunch|test preparation course|math score|reading score|writing score|ntile|
+------+--------------+---------------------------+------------+-----------------------+----------+-------------+-------------+-----+
|female|       group D|         associate's degree|free/reduced|                   none|        26|           31|           38|    1|
|female|       group A|         associate's degree|free/reduced|                   none|        37|           57|           56|    1|
|female|       group C|         associate's degree|    standard|                   none|        39|           64|           57|    1|
|  male|       group D|         associate's degree|    standard|                   none|        40|           52|           43|    1|
|female|       group B|         associate's degree|    standar

#### pyspark.sql.functions.percent_rank() → pyspark.sql.column.Column
Window function: returns the relative rank (i.e. percentile) of rows within a window partition.

In [229]:
df = spark.createDataFrame([1, 1, 2, 3, 3, 4], types.IntegerType())
w = Window.orderBy("value")
df.withColumn("pr", percent_rank().over(w)).show()

23/10/22 21:15:01 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/10/22 21:15:01 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/10/22 21:15:01 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+-----+---+
|value| pr|
+-----+---+
|    1|0.0|
|    1|0.0|
|    2|0.4|
|    3|0.6|
|    3|0.6|
|    4|1.0|
+-----+---+



23/10/22 21:15:01 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/10/22 21:15:01 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


In [241]:
w = Window.partitionBy("parental level of education").orderBy("math score")
students.withColumn("pr", percent_rank().over(w)).select("math score", "pr").distinct().show(100)

+----------+--------------------+
|math score|                  pr|
+----------+--------------------+
|        44| 0.05128205128205128|
|        49| 0.08547008547008547|
|        58| 0.27586206896551724|
|        77|                0.76|
|        90|  0.9095022624434389|
|        53|  0.1452991452991453|
|        79|  0.6379310344827587|
|        60| 0.28444444444444444|
|        59|  0.3089887640449438|
|        35|  0.0449438202247191|
|        59| 0.37435897435897436|
|        92|  0.9775280898876404|
|        61| 0.23076923076923078|
|        41| 0.07865168539325842|
|        51| 0.12669683257918551|
|        57| 0.24434389140271492|
|        63| 0.49743589743589745|
|        69|  0.6461538461538462|
|        53|  0.1724137931034483|
|        69|  0.5288888888888889|
|        58| 0.28054298642533937|
|        29|                 0.0|
|        60|  0.3595505617977528|
|        61|   0.334841628959276|
|        95|  0.9683257918552036|
|        74|                 0.8|
|        88|  

#### pyspark.sql.functions.rank() → pyspark.sql.column.Column
Window function: returns the rank of rows within a window partition.

The difference between rank and dense_rank is that dense_rank leaves no gaps in ranking sequence when there are ties. That is, if you were ranking a competition using dense_rank and had three people tie for second place, you would say that all three were in second place and that the next person came in third. Rank would give me sequential numbers, making the person that came in third place (after the ties) would register as coming in fifth.

This is equivalent to the RANK function in SQL.

In [237]:
df = spark.createDataFrame([1, 1, 2, 3, 3, 4], types.IntegerType())
w = Window.orderBy("value")
df.withColumn("drank", rank().over(w)).show()

23/10/22 21:20:00 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/10/22 21:20:00 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/10/22 21:20:00 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+-----+-----+
|value|drank|
+-----+-----+
|    1|    1|
|    1|    1|
|    2|    3|
|    3|    4|
|    3|    4|
|    4|    6|
+-----+-----+



In [242]:
w = Window.partitionBy("parental level of education").orderBy("math score")
students.withColumn("drank", rank().over(w)).select("math score", "drank").distinct().show(100)

+----------+-----+
|math score|drank|
+----------+-----+
|        68|   57|
|        73|  152|
|        47|   24|
|        72|  131|
|        84|  187|
|        90|  202|
|        92|  211|
|        72|   69|
|        64|   96|
|        58|   67|
|        68|  116|
|        78|  148|
|        89|  106|
|        42|   10|
|        76|  168|
|        64|  102|
|        81|  169|
|        66|   48|
|        48|   30|
|        50|   37|
|        41|    8|
|        55|   20|
|        98|  221|
|        81|   94|
|        39|    3|
|        88|  200|
|        60|   80|
|        69|   61|
|        76|  148|
|        27|    4|
|        99|  223|
|        52|   27|
|        62|   32|
|        70|   27|
|        68|  114|
|        39|   13|
|        53|   18|
|        64|   41|
|        78|   84|
|        75|  137|
|        51|   14|
|        75|   75|
|        87|   51|
|        66|  106|
|        53|   38|
|        90|   54|
|        84|  163|
|        71|   66|
|        92|  109|
|        69|

#### pyspark.sql.functions.row_number() → pyspark.sql.column.Column
Window function: returns a sequential number starting at 1 within a window partition.

In [240]:
df = spark.range(3)
w = Window.orderBy(df.id.desc())
df.withColumn("desc_order", row_number().over(w)).show()

+---+----------+
| id|desc_order|
+---+----------+
|  2|         1|
|  1|         2|
|  0|         3|
+---+----------+



23/10/22 21:21:47 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/10/22 21:21:47 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/10/22 21:21:47 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/10/22 21:21:47 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/10/22 21:21:47 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


In [246]:
w = Window.partitionBy("parental level of education").orderBy("math score")
students.withColumn("desc_order", row_number().over(w)).select("math score", "desc_order").distinct().limit(20).orderBy(asc("desc_order")).show()

+----------+----------+
|math score|desc_order|
+----------+----------+
|        32|         8|
|        47|        24|
|        53|        49|
|        68|        57|
|        68|        60|
|        69|        62|
|        72|        69|
|        61|        77|
|        62|        78|
|        63|        81|
|        60|        84|
|        80|        93|
|        69|       112|
|        72|       131|
|        73|       152|
|        73|       153|
|        84|       185|
|        84|       187|
|        90|       202|
|        92|       211|
+----------+----------+



## [Sort Functions](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/functions.html#sort-functions)

#### pyspark.sql.functions.asc(col: ColumnOrName) → pyspark.sql.column.Column
Returns a sort expression based on the ascending order of the given column name.

In [182]:
df = spark.range(5)
df = df.sort(desc("id"))
df.show()

+---+
| id|
+---+
|  4|
|  3|
|  2|
|  1|
|  0|
+---+



In [183]:
df.orderBy(asc("id")).show()

+---+
| id|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
+---+



In [189]:
students.orderBy(asc("math score")).toPandas()

gender race/ethnicity parental level of education         lunch  \
0    female        group C            some high school  free/reduced   
1    female        group B                 high school  free/reduced   
2    female        group B            some high school  free/reduced   
3    female        group B                some college      standard   
4    female        group C                some college  free/reduced   
..      ...            ...                         ...           ...   
995  female        group E           bachelor's degree      standard   
996    male        group A                some college      standard   
997    male        group D                some college      standard   
998    male        group E           bachelor's degree      standard   
999  female        group E          associate's degree      standard   

    test preparation course  math score  reading score  writing score  
0                      none           0             17             10  
1                      none           8             24             23  
2                      none          18             32             28  
3                      none          19             38             32  
4                      none          22             39             33  
..                      ...         ...            ...            ...  
995                    none         100            100            100  
996               completed         100             96             86  
997               completed         100             97             99  
998               completed         100            100            100  
999                    none         100            100            100  

[1000 rows x 8 columns]

#### pyspark.sql.functions.asc_nulls_first(col: ColumnOrName) → pyspark.sql.column.Column
Returns a sort expression based on the ascending order of the given column name, and null values return before non-null values.

In [184]:
df1 = spark.createDataFrame([(1, "Bob"),
                             (0, None),
                             (2, "Alice")], ["age", "name"])
df1.sort(asc_nulls_first(df1.name)).show()

+---+-----+
|age| name|
+---+-----+
|  0| NULL|
|  2|Alice|
|  1|  Bob|
+---+-----+



#### pyspark.sql.functions.asc_nulls_last(col: ColumnOrName) → pyspark.sql.column.Column
Returns a sort expression based on the ascending order of the given column name, and null values appear after non-null values.

In [185]:
df1 = spark.createDataFrame([(0, None),
                             (1, "Bob"),
                             (2, "Alice")], ["age", "name"])
df1.sort(asc_nulls_last(df1.name)).show()

+---+-----+
|age| name|
+---+-----+
|  2|Alice|
|  1|  Bob|
|  0| NULL|
+---+-----+



In [192]:
tour.orderBy(asc("Value")).show()

+----------------+------+--------------------+--------------------+-----+
|     Player Name|Season|           Statistic|            Variable|Value|
+----------------+------+--------------------+--------------------+-----+
|   Jordan Spieth|  2018| Par 4 Eagle Leaders|Par 4 Eagle Leade...| NULL|
|   Gary Woodland|  2016|% of Potential Pt...|% of Potential Pt...| NULL|
|Brendon de Jonge|  2010|FedExCup Season P...|FedExCup Season P...| NULL|
|      Jon Curran|  2016|% of Potential Pt...|% of Potential Pt...| NULL|
| Charley Hoffman|  2017|FedExCup Season P...|FedExCup Season P...| NULL|
| David Lingmerth|  2016|% of Potential Pt...|% of Potential Pt...| NULL|
|      Charlie Wi|  2010|FedExCup Season P...|FedExCup Season P...| NULL|
|    Patrick Reed|  2016|% of Potential Pt...|% of Potential Pt...| NULL|
|    Bubba Watson|  2013|FedExCup Season P...|FedExCup Season P...| NULL|
|    Zach Johnson|  2016|% of Potential Pt...|% of Potential Pt...| NULL|
|      Paul Casey|  2010|FedExCup Seas

In [193]:
tour.orderBy(asc_nulls_last("Value")).show()

+--------------+------+--------------------+--------------------+----------+
|   Player Name|Season|           Statistic|            Variable|     Value|
+--------------+------+--------------------+--------------------+----------+
|   Chris Riley|  2010|Percentage of Ava...|Percentage of Ava...|$1,001,580|
|   Chris Riley|  2010|Percentage of pot...|Percentage of pot...|$1,001,581|
|   Chris Riley|  2010|Money per Event L...|Money per Event L...|$1,001,581|
|Steve Stricker|  2017|      Official Money|Official Money - ...|$1,002,036|
|Steve Stricker|  2017|Percentage of Ava...|Percentage of Ava...|$1,002,036|
|Steve Stricker|  2017|Money per Event L...|Money per Event L...|$1,002,036|
|Steve Stricker|  2017|Percentage of pot...|Percentage of pot...|$1,002,036|
|  Robert Streb|  2016|Percentage of Ava...|Percentage of Ava...|$1,003,359|
|  Robert Streb|  2016|Percentage of pot...|Percentage of pot...|$1,003,362|
|  Robert Streb|  2016|Money per Event L...|Money per Event L...|$1,003,362|

#### pyspark.sql.functions.desc(col: ColumnOrName) → pyspark.sql.column.Column
Returns a sort expression based on the descending order of the given column name.

In [187]:
spark.range(5).orderBy(desc("id")).show()

+---+
| id|
+---+
|  4|
|  3|
|  2|
|  1|
|  0|
+---+



In [191]:
tour.orderBy(desc("Value")).show()

+------------------+------+--------------------+--------------------+--------------------+
|       Player Name|Season|           Statistic|            Variable|               Value|
+------------------+------+--------------------+--------------------+--------------------+
|    Soren Kjeldsen|  2016|        Lowest Round|Lowest Round - (T...|     the Memorial/Mu|
|      Jason Dufner|  2017|        Lowest Round|Lowest Round - (T...|     the Memorial/Mu|
|       Vijay Singh|  2018|        Lowest Round|Lowest Round - (T...|     the Memorial/Mu|
|     Scott Gregory|  2017|        Lowest Round|Lowest Round - (T...|     the Memorial/Mu|
|      Ryan Ruffels|  2016|        Lowest Round|Lowest Round - (T...|     the Memorial/Mu|
|       Andrew Dorn|  2018|        Lowest Round|Lowest Round - (T...|     the Memorial/Mu|
|         John Hahn|  2016|        Lowest Round|Lowest Round - (T...|     the Memorial/Mu|
|       Kenny Perry|  2018|        Lowest Round|Lowest Round - (T...|     the Memorial/Mu|

#### pyspark.sql.functions.desc_nulls_first(col: ColumnOrName) → pyspark.sql.column.Column
Returns a sort expression based on the descending order of the given column name, and null values appear before non-null values.

In [195]:
df1 = spark.createDataFrame([(0, None),
                             (1, "Bob"),
                             (2, "Alice")], ["age", "name"])
df1.sort(desc_nulls_first(df1.name)).show()

+---+-----+
|age| name|
+---+-----+
|  0| NULL|
|  1|  Bob|
|  2|Alice|
+---+-----+



#### pyspark.sql.functions.desc_nulls_last(col: ColumnOrName) → pyspark.sql.column.Column
Returns a sort expression based on the descending order of the given column name, and null values appear after non-null values.

In [196]:
df1 = spark.createDataFrame([(0, None),
                             (1, "Bob"),
                             (2, "Alice")], ["age", "name"])
df1.sort(desc_nulls_last(df1.name)).show()

+---+-----+
|age| name|
+---+-----+
|  1|  Bob|
|  2|Alice|
|  0| NULL|
+---+-----+



In [197]:
tour.orderBy(desc_nulls_last("Value")).show()

+------------------+------+--------------------+--------------------+--------------------+
|       Player Name|Season|           Statistic|            Variable|               Value|
+------------------+------+--------------------+--------------------+--------------------+
|      Jason Dufner|  2017|        Lowest Round|Lowest Round - (T...|     the Memorial/Mu|
|       Vijay Singh|  2018|        Lowest Round|Lowest Round - (T...|     the Memorial/Mu|
|    Soren Kjeldsen|  2016|        Lowest Round|Lowest Round - (T...|     the Memorial/Mu|
|       Andrew Dorn|  2018|        Lowest Round|Lowest Round - (T...|     the Memorial/Mu|
|     Scott Gregory|  2017|        Lowest Round|Lowest Round - (T...|     the Memorial/Mu|
|       Kenny Perry|  2018|        Lowest Round|Lowest Round - (T...|     the Memorial/Mu|
|      Ryan Ruffels|  2016|        Lowest Round|Lowest Round - (T...|     the Memorial/Mu|
|       Harry Ellis|  2018|        Lowest Round|Lowest Round - (T...|     the Memorial/Mu|

## [String Functions](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/functions.html#string-functions)

#### pyspark.sql.functions.ascii(col: ColumnOrName) → pyspark.sql.column.Column
Computes the numeric value of the first character of the string column.

In [248]:
df = spark.createDataFrame(["Spark", "PySpark", "Pandas API"], "STRING")
df.select(ascii("value")).show()

+------------+
|ascii(value)|
+------------+
|          83|
|          80|
|          80|
+------------+



#### pyspark.sql.functions.base64(col: ColumnOrName) → pyspark.sql.column.Column
Computes the BASE64 encoding of a binary column and returns it as a string column.

In [250]:
df = spark.createDataFrame(["Spark", "PySpark", "Pandas API"], "STRING")
df.select(base64("value")).show()

+----------------+
|   base64(value)|
+----------------+
|        U3Bhcms=|
|    UHlTcGFyaw==|
|UGFuZGFzIEFQSQ==|
+----------------+



#### pyspark.sql.functions.bit_length(col: ColumnOrName) → pyspark.sql.column.Column
Calculates the bit length for the specified string column.

In [257]:
df = spark.createDataFrame([('cat',), ( '🐈',), ( 'кіт',)], ['cat'])
df.select(bit_length('cat')).collect()

[Row(bit_length(cat)=24), Row(bit_length(cat)=32), Row(bit_length(cat)=48)]

#### pyspark.sql.functions.btrim(str: ColumnOrName, trim: Optional[ColumnOrName] = None) → pyspark.sql.column.Column
Remove the leading and trailing trim characters from str.

In [262]:
df = spark.createDataFrame([("SSparkSQLS", "SL", )], ['a', 'b'])
df.select(btrim(df.a, df.b).alias('r')).collect()

[Row(r='parkSQ')]

In [260]:
df = spark.createDataFrame([("    SparkSQL   ",)], ['a'])
df.select(btrim(df.a).alias('r')).collect()

[Row(r='SparkSQL')]

#### pyspark.sql.functions.char(col: ColumnOrName) → pyspark.sql.column.Column
Returns the ASCII character having the binary equivalent to col. If col is larger than 256 the result is equivalent to char(col % 256)

In [263]:
spark.range(1).select(char(lit(65))).show()

+--------+
|char(65)|
+--------+
|       A|
+--------+



#### pyspark.sql.functions.character_length(str: ColumnOrName) → pyspark.sql.column.Column
Returns the character length of string data or number of bytes of binary data. The length of string data includes the trailing spaces. The length of binary data includes binary zeros.

In [264]:
spark.range(1).select(sf.character_length(sf.lit("SparkSQL"))).show()

+--------------------------+
|character_length(SparkSQL)|
+--------------------------+
|                         8|
+--------------------------+



#### pyspark.sql.functions.char_length(str: ColumnOrName) → pyspark.sql.column.Column
Returns the character length of string data or number of bytes of binary data. The length of string data includes the trailing spaces. The length of binary data includes binary zeros.

In [265]:
spark.range(1).select(char_length(lit("SparkSQL"))).show()

+---------------------+
|char_length(SparkSQL)|
+---------------------+
|                    8|
+---------------------+



#### pyspark.sql.functions.concat_ws(sep: str, *cols: ColumnOrName) → pyspark.sql.column.Column¶
Concatenates multiple input string columns together into a single string column, using the given separator.

In [266]:
df = spark.createDataFrame([('abcd','123')], ['s', 'd'])
df.select(concat_ws('-', df.s, df.d).alias('s')).collect()

[Row(s='abcd-123')]

#### pyspark.sql.functions.contains(left: ColumnOrName, right: ColumnOrName) → pyspark.sql.column.Column
Returns a boolean. The value is True if right is found inside left. Returns NULL if either input expression is NULL. Otherwise, returns False. Both left or right must be of STRING or BINARY type.

Parameters:
- left: Column or str | 
The input column or strings to check, may be NULL.
- right: Column or str | 
The input column or strings to find, may be NULL.

In [267]:
df = spark.createDataFrame([("Spark SQL", "Spark")], ['a', 'b'])
df.select(contains(df.a, df.b).alias('r')).collect()

[Row(r=True)]

In [268]:
df = spark.createDataFrame([("414243", "4243",)], ["c", "d"])
df = df.select(to_binary("c").alias("c"), to_binary("d").alias("d"))
df.printSchema()

root
 |-- c: binary (nullable = true)
 |-- d: binary (nullable = true)



In [269]:
df.select(contains("c", "d"), contains("d", "c")).show()

+--------------+--------------+
|contains(c, d)|contains(d, c)|
+--------------+--------------+
|          true|         false|
+--------------+--------------+



#### pyspark.sql.functions.decode(col: ColumnOrName, charset: str) → pyspark.sql.column.Column
Computes the first argument into a string from a binary using the provided character set (one of ‘US-ASCII’, ‘ISO-8859-1’, ‘UTF-8’, ‘UTF-16BE’, ‘UTF-16LE’, ‘UTF-16’).

In [270]:
df = spark.createDataFrame([('abcd',)], ['a'])
df.select(decode("a", "UTF-8")).show()

+----------------+
|decode(a, UTF-8)|
+----------------+
|            abcd|
+----------------+



In [277]:
df = spark.createDataFrame([('кіт кит дельфін',)], ['a'])
df.select(decode("a", "UTF-8")).show()

+----------------+
|decode(a, UTF-8)|
+----------------+
| кіт кит дельфін|
+----------------+



#### pyspark.sql.functions.elt(*inputs: ColumnOrName) → pyspark.sql.column.Column
Returns the n-th input, e.g., returns input2 when n is 2. The function returns NULL if the index exceeds the length of the array and spark.sql.ansi.enabled is set to false. If spark.sql.ansi.enabled is set to true, it throws ArrayIndexOutOfBoundsException for invalid indices.

In [278]:
df = spark.createDataFrame([(1, "scala", "java")], ['a', 'b', 'c'])
df.select(elt(df.a, df.b, df.c).alias('r')).collect()

[Row(r='scala')]

In [281]:
df = spark.createDataFrame([(1, "scala", "java")], ['a', 'b', 'c'])
df.select(elt(df.a, df.b, df.c).alias('r')).collect()

[Row(r='scala')]

In [282]:
df = spark.createDataFrame([("ggg", "scala", "java")], ['a', 'b', 'c'])
df.select(elt(lit(2), df.b, df.c).alias('r')).collect()

[Row(r='java')]

In [286]:
df.select(elt(lit(0), df.b, df.c).alias('r'), elt(lit(3), df.b, df.c).alias('r')).collect()

[Row(r=None, r=None)]

#### pyspark.sql.functions.encode(col: ColumnOrName, charset: str) → pyspark.sql.column.Column
Computes the first argument into a binary from a string using the provided character set (one of ‘US-ASCII’, ‘ISO-8859-1’, ‘UTF-8’, ‘UTF-16BE’, ‘UTF-16LE’, ‘UTF-16’).

In [287]:
df = spark.createDataFrame([('abcd',)], ['c'])
df.select(encode("c", "UTF-8")).show()

+----------------+
|encode(c, UTF-8)|
+----------------+
|   [61 62 63 64]|
+----------------+



In [294]:
df = spark.createDataFrame([('кіт кит дельфін',)], ['c'])
df = df.select(encode("c", "UTF-16").alias("encode"))
df.show(truncate=False)
df.select(decode("encode", "UTF-16")).show()

+-------------------------------------------------------------------------------------------------+
|encode                                                                                           |
+-------------------------------------------------------------------------------------------------+
|[FE FF 04 3A 04 56 04 42 00 20 04 3A 04 38 04 42 00 20 04 34 04 35 04 3B 04 4C 04 44 04 56 04 3D]|
+-------------------------------------------------------------------------------------------------+

+----------------------+
|decode(encode, UTF-16)|
+----------------------+
|       кіт кит дельфін|
+----------------------+



#### pyspark.sql.functions.endswith(str: ColumnOrName, suffix: ColumnOrName) → pyspark.sql.column.Column
Returns a boolean. The value is True if str ends with suffix. Returns NULL if either input expression is NULL. Otherwise, returns False. Both str or suffix must be of STRING or BINARY type.

In [295]:
df = spark.createDataFrame([("Spark SQL", "Spark",)], ["a", "b"])
df.select(endswith(df.a, df.b).alias('r')).collect()

[Row(r=False)]

In [296]:
df = spark.createDataFrame([("414243", "4243",)], ["e", "f"])
df = df.select(to_binary("e").alias("e"), to_binary("f").alias("f"))
df.printSchema()

root
 |-- e: binary (nullable = true)
 |-- f: binary (nullable = true)



In [297]:
df.select(endswith("e", "f"), endswith("f", "e")).show()

+--------------+--------------+
|endswith(e, f)|endswith(f, e)|
+--------------+--------------+
|          true|         false|
+--------------+--------------+



#### pyspark.sql.functions.find_in_set(str: ColumnOrName, str_array: ColumnOrName) → pyspark.sql.column.Column
Returns the index (1-based) of the given string (str) in the comma-delimited list (strArray). Returns 0, if the string was not found or if the given string (str) contains a comma.

In [298]:
df = spark.createDataFrame([("ab", "abc,b,ab,c,def")], ['a', 'b'])
df.select(find_in_set(df.a, df.b).alias('r')).collect()

[Row(r=3)]

#### pyspark.sql.functions.format_number(col: ColumnOrName, d: int) → pyspark.sql.column.Column
Formats the number X to a format like ‘#,–#,–#.–’, rounded to d decimal places with HALF_EVEN round mode, and returns the result as a string.

In [299]:
spark.createDataFrame([(5,)], ['a']).select(format_number('a', 4).alias('v')).collect()

[Row(v='5.0000')]

#### pyspark.sql.functions.format_string(format: str, *cols: ColumnOrName) → pyspark.sql.column.Column
Formats the arguments in printf-style and returns the result as a string column.

In [300]:
df = spark.createDataFrame([(5, "hello")], ['a', 'b'])
df.select(format_string('%d %s', df.a, df.b).alias('v')).collect()

[Row(v='5 hello')]

#### pyspark.sql.functions.ilike(str: ColumnOrName, pattern: ColumnOrName, escapeChar: Optional[Column] = None) → pyspark.sql.column.Column
Returns true if str matches pattern with escape case-insensitively, null if any arguments are null, false otherwise. The default escape character is the ‘’.

Parameters: 
- str: Column or str | 
A string.
- pattern: Column or str |
A string. The pattern is a string which is matched literally, with exception to the following special symbols: _ matches any one character in the input (similar to . in posix regular expressions) % matches zero or more characters in the input (similar to .* in posix regular expressions) Since Spark 2.0, string literals are unescaped in our SQL parser. For example, in order to match “bc”, the pattern should be “abc”. When SQL config ‘spark.sql.parser.escapedStringLiterals’ is enabled, it falls back to Spark 1.6 behavior regarding string literal parsing. For example, if the config is enabled, the pattern to match “bc” should be “bc”.
- escape: Column |
An character added since Spark 3.0. The default escape character is the ‘’. If an escape character precedes a special symbol or another escape character, the following character is matched literally. It is invalid to escape any other character.

In [301]:
df = spark.createDataFrame([("Spark", "_park")], ['a', 'b'])
df.select(ilike(df.a, df.b).alias('r')).collect()

[Row(r=True)]

In [302]:
df = spark.createDataFrame(
    [("%SystemDrive%/Users/John", "/%SystemDrive/%//Users%")],
    ['a', 'b']
)
df.select(ilike(df.a, df.b, lit('/')).alias('r')).collect()

[Row(r=True)]

#### pyspark.sql.functions.initcap(col: ColumnOrName) → pyspark.sql.column.Column
Translate the first letter of each word to upper case in the sentence.

In [304]:
spark.createDataFrame([('ab cd',)], ['a']).select(initcap("a").alias('v')).collect()

[Row(v='Ab Cd')]

#### pyspark.sql.functions.instr(str: ColumnOrName, substr: str) → pyspark.sql.column.Column
Locate the position of the first occurrence of substr column in the given string. Returns null if either of the arguments are null.

##### Notes

The position is not zero based, but 1 based index. Returns 0 if substr could not be found in str.

In [306]:
df = spark.createDataFrame([('abcd',)], ['s',])
df.select(instr(df.s, 'b').alias('s')).collect()

[Row(s=2)]

#### pyspark.sql.functions.lcase(str: ColumnOrName) → pyspark.sql.column.Column
Returns str with all characters changed to lowercase.

In [307]:
spark.range(1).select(sf.lcase(sf.lit("Spark"))).show()

+------------+
|lcase(Spark)|
+------------+
|       spark|
+------------+



#### pyspark.sql.functions.length(col: ColumnOrName) → pyspark.sql.column.Column
Computes the character length of string data or number of bytes of binary data. The length of character data includes the trailing spaces. The length of binary data includes binary zeros.

In [308]:
spark.createDataFrame([('ABC ',)], ['a']).select(length('a').alias('length')).collect()

[Row(length=4)]

#### pyspark.sql.functions.like(str: ColumnOrName, pattern: ColumnOrName, escapeChar: Optional[Column] = None) → pyspark.sql.column.Column
Returns true if str matches pattern with escape, null if any arguments are null, false otherwise. The default escape character is the ‘’.

Parameters:
- str: Column or str |
A string.
- pattern: Column or str | 
A string. The pattern is a string which is matched literally, with exception to the following special symbols: _ matches any one character in the input (similar to . in posix regular expressions) % matches zero or more characters in the input (similar to .* in posix regular expressions) Since Spark 2.0, string literals are unescaped in our SQL parser. For example, in order to match “bc”, the pattern should be “abc”. When SQL config ‘spark.sql.parser.escapedStringLiterals’ is enabled, it falls back to Spark 1.6 behavior regarding string literal parsing. For example, if the config is enabled, the pattern to match “bc” should be “bc”.
- escape: Column |
An character added since Spark 3.0. The default escape character is the ‘’. If an escape character precedes a special symbol or another escape character, the following character is matched literally. It is invalid to escape any other character.

In [309]:
df = spark.createDataFrame([("Spark", "_park")], ['a', 'b'])
df.select(like(df.a, df.b).alias('r')).collect()

[Row(r=True)]

In [310]:
df = spark.createDataFrame(
    [("%SystemDrive%/Users/John", "/%SystemDrive/%//Users%")],
    ['a', 'b']
)
df.select(like(df.a, df.b, lit('/')).alias('r')).collect()

[Row(r=True)]

#### pyspark.sql.functions.lower(col: ColumnOrName) → pyspark.sql.column.Column¶
Converts a string expression to lower case.

In [311]:
df = spark.createDataFrame(["Spark", "PySpark", "Pandas API"], "STRING")
df.select(lower("value")).show()

+------------+
|lower(value)|
+------------+
|       spark|
|     pyspark|
|  pandas api|
+------------+



#### pyspark.sql.functions.left(str: ColumnOrName, len: ColumnOrName) → pyspark.sql.column.Column
    Returns the leftmost len`(`len can be string type) characters from the string str, if len is less or equal than 0 the result is an empty string.

In [312]:
df = spark.createDataFrame([("Spark SQL", 3,)], ['a', 'b'])
df.select(left(df.a, df.b).alias('r')).collect()

[Row(r='Spa')]

#### pyspark.sql.functions.levenshtein(left: ColumnOrName, right: ColumnOrName, threshold: Optional[int] = None) → pyspark.sql.column.Column
Computes the Levenshtein distance of the two given strings.

Parameters:
- leftColumn or str
first column value.
- right: Column or str | 
second column value.
- threshold: int, optional | 
if set when the levenshtein distance of the two given strings less than or equal to a given threshold then return result distance, or -1

Returns – Column | Levenshtein distance as integer value.

In [313]:
df0 = spark.createDataFrame([('kitten', 'sitting',)], ['l', 'r'])
df0.select(levenshtein('l', 'r').alias('d')).collect()

[Row(d=3)]

In [314]:
df0.select(levenshtein('l', 'r', 2).alias('d')).collect()

[Row(d=-1)]

#### pyspark.sql.functions.locate(substr: str, str: ColumnOrName, pos: int = 1) → pyspark.sql.column.Column
Locate the position of the first occurrence of substr in a string column, after position pos.

##### Notes

The position is not zero based, but 1 based index. Returns 0 if substr could not be found in str.

In [315]:
df = spark.createDataFrame([('abcd',)], ['s',])
df.select(locate('b', df.s, 1).alias('s')).collect()

[Row(s=2)]

#### pyspark.sql.functions.lpad(col: ColumnOrName, len: int, pad: str) → pyspark.sql.column.Column
Left-pad the string column to width len with pad.

In [316]:
df = spark.createDataFrame([('abcd',)], ['s',])
df.select(lpad(df.s, 6, '#').alias('s')).collect()

[Row(s='##abcd')]

In [318]:
tour.limit(10).select(lpad(tour["Player name"], 20, '~').alias('s')).show()

+--------------------+
|                   s|
+--------------------+
|~~~~~Robert Garrigus|
|~~~~~~~~Bubba Watson|
|~~~~~~Dustin Johnson|
|~~~~~Brett Wetterich|
|~~~~~~~~~J.B. Holmes|
|~~~~~~~~~~~John Daly|
|~~~~~~~Graham DeLaet|
|~~~~~~~Angel Cabrera|
|~~~~~~Charles Warren|
|~~~~~~~~~D.J. Trahan|
+--------------------+



#### pyspark.sql.functions.ltrim(col: ColumnOrName) → pyspark.sql.column.Column
Trim the spaces from left end for the specified string value.

In [319]:
df = spark.createDataFrame(["   Spark", "Spark  ", " Spark"], "STRING")
df.select(ltrim("value").alias("r")).withColumn("length", length("r")).show()

+-------+------+
|      r|length|
+-------+------+
|  Spark|     5|
|Spark  |     7|
|  Spark|     5|
+-------+------+



#### pyspark.sql.functions.mask(col: ColumnOrName, upperChar: Optional[ColumnOrName] = None, lowerChar: Optional[ColumnOrName] = None, digitChar: Optional[ColumnOrName] = None, otherChar: Optional[ColumnOrName] = None) → pyspark.sql.column.Column
Masks the given string value. This can be useful for creating copies of tables with sensitive information removed.

Parameters:

    col: :class:`~pyspark.sql.Column` or str
        target column to compute on.

    upperChar: :class:`~pyspark.sql.Column` or str
        character to replace upper-case characters with. Specify NULL to retain original character.

    lowerChar: :class:`~pyspark.sql.Column` or str
        character to replace lower-case characters with. Specify NULL to retain original character.

    digitChar: :class:`~pyspark.sql.Column` or str
        character to replace digit characters with. Specify NULL to retain original character.

    otherChar: :class:`~pyspark.sql.Column` or str
        character to replace all other characters with. Specify NULL to retain original character.

Returns | Column

In [320]:
df = spark.createDataFrame([("AbCD123-@$#",), ("abcd-EFGH-8765-4321",)], ['data'])
df.select(mask(df.data).alias('r')).collect()

[Row(r='XxXXnnn-@$#'), Row(r='xxxx-XXXX-nnnn-nnnn')]

In [321]:
df.select(mask(df.data, lit('Y')).alias('r')).collect()

[Row(r='YxYYnnn-@$#'), Row(r='xxxx-YYYY-nnnn-nnnn')]

In [322]:
df.select(mask(df.data, lit('Y'), lit('y')).alias('r')).collect()

[Row(r='YyYYnnn-@$#'), Row(r='yyyy-YYYY-nnnn-nnnn')]

In [323]:
df.select(mask(df.data, lit('Y'), lit('y'), lit('d')).alias('r')).collect()

[Row(r='YyYYddd-@$#'), Row(r='yyyy-YYYY-dddd-dddd')]

In [324]:
df.select(mask(df.data, lit('Y'), lit('y'), lit('d'), lit('*')).alias('r')).collect()

[Row(r='YyYYddd****'), Row(r='yyyy*YYYY*dddd*dddd')]

In [326]:
df = spark.createDataFrame([("AbCD123-@$#",),
                            ("abcd-EFGH-8765-4321",),
                            ("1234-5678-8765-4321",),
                            ("12345678",),
                            ("Абаба Галамага",),
                            ("Тест_1234_!'№;_test",),
                            ("k[MY0&[&jwts",),
                           ], ['data'])
df.select(mask(df.data).alias('r')).collect()

[Row(r='XxXXnnn-@$#'),
 Row(r='xxxx-XXXX-nnnn-nnnn'),
 Row(r='nnnn-nnnn-nnnn-nnnn'),
 Row(r='nnnnnnnn'),
 Row(r='Xxxxx Xxxxxxxx'),
 Row(r="Xxxx_nnnn_!'№;_xxxx"),
 Row(r='x[XXn&[&xxxx')]

#### pyspark.sql.functions.octet_length(col: ColumnOrName) → pyspark.sql.column.Column
Calculates the byte length for the specified string column.

In [327]:
spark.createDataFrame([('cat',), ( '🐈',)], ['cat']) \
     .select(octet_length('cat')).collect()

[Row(octet_length(cat)=3), Row(octet_length(cat)=4)]

#### pyspark.sql.functions.parse_url(url: ColumnOrName, partToExtract: ColumnOrName, key: Optional[ColumnOrName] = None) → pyspark.sql.column.Column
Extracts a part from a URL.

    'HOST': Домен (наприклад, www.example.com).
    'PATH': Шлях (наприклад, /page1).
    'QUERY': Рядок запиту (наприклад, ?param1=value1&param2=value2).
    'SCHEME': Протокол (наприклад, http).
    'FILE': Шлях до файлу.
    'REF': Фрагмент (якщо він вказаний у URL).

In [328]:
df = spark.createDataFrame(
    [("http://spark.apache.org/path?query=1", "QUERY", "query",)],
    ["a", "b", "c"]
)
df.select(parse_url(df.a, df.b, df.c).alias('r')).collect()

[Row(r='1')]

In [329]:
df.select(parse_url(df.a, df.b).alias('r')).collect()

[Row(r='query=1')]

In [336]:
df.select(parse_url(df.a, lit("PATH")).alias('r')).collect()

[Row(r='/path')]

In [340]:
df.select(parse_url(df.a, lit("HOST")).alias('r')).collect()

[Row(r='spark.apache.org')]

#### pyspark.sql.functions.position(substr: ColumnOrName, str: ColumnOrName, start: Optional[ColumnOrName] = None) → pyspark.sql.column.Column
Returns the position of the first occurrence of substr in str after position start. The given start and return value are 1-based.

In [344]:
spark.createDataFrame(
    [("bar", "foobarbar", 5,)], ["a", "b", "c"]
).select(position("a", "b", "c")).show()

+-----------------+
|position(a, b, c)|
+-----------------+
|                7|
+-----------------+



In [345]:
spark.createDataFrame(
    [("bar", "foobarbar", 5,)], ["a", "b", "c"]
).select(position("a", "b")).show()

+-----------------+
|position(a, b, 1)|
+-----------------+
|                4|
+-----------------+



#### pyspark.sql.functions.printf(format: ColumnOrName, *cols: ColumnOrName) → pyspark.sql.column.Column
Formats the arguments in printf-style and returns the result as a string column.

In [346]:
spark.createDataFrame(
    [("aa%d%sqwe", 123, "cc",)], ["a", "b", "c"]
).select(printf("a", "b", "c")).show()

+---------------+
|printf(a, b, c)|
+---------------+
|     aa123ccqwe|
+---------------+



#### pyspark.sql.functions.rlike(str: ColumnOrName, regexp: ColumnOrName) → pyspark.sql.column.Column
Returns true if str matches the Java regex regexp, or false otherwise.

In [347]:
df = spark.createDataFrame([("1a 2b 14m", r"(\d+)")], ["str", "regexp"])
df.select(rlike('str', lit(r'(\d+)')).alias('d')).collect()

[Row(d=True)]

In [348]:
df.select(rlike('str', lit(r'\d{2}b')).alias('d')).collect()

[Row(d=False)]

In [349]:
df.select(rlike("str", col("regexp")).alias('d')).collect()

[Row(d=True)]

#### pyspark.sql.functions.regexp(str: ColumnOrName, regexp: ColumnOrName) → pyspark.sql.column.Column
Returns true if str matches the Java regex regexp, or false otherwise.

Parameters:
- str: Column or str | 
target column to work on.
- regexp: Column or str | 
regex pattern to apply.

Returns – Column | true if str matches a Java regex, or false otherwise.

In [351]:
spark.createDataFrame(
    [("1a 2b 14m", r"(\d+)")], ["str", "regexp"]
).select(regexp('str', lit(r'(\d+)'))).show()

+------------------+
|REGEXP(str, (\d+))|
+------------------+
|              true|
+------------------+



In [352]:
spark.createDataFrame(
    [("1a 2b 14m", r"(\d+)")], ["str", "regexp"]
).select(regexp('str', lit(r'\d{2}b'))).show()

+-------------------+
|REGEXP(str, \d{2}b)|
+-------------------+
|              false|
+-------------------+



In [353]:
spark.createDataFrame(
    [("1a 2b 14m", r"(\d+)")], ["str", "regexp"]
).select(regexp('str', col("regexp"))).show()

+-------------------+
|REGEXP(str, regexp)|
+-------------------+
|               true|
+-------------------+



## [Bitwise Functions](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/functions.html#bitwise-functions)

#### pyspark.sql.functions.bit_count(col: ColumnOrName) → pyspark.sql.column.Column
Returns the number of bits that are set in the argument expr as an unsigned 64-bit integer, or NULL if the argument is NULL.

In [177]:
df = spark.createDataFrame([[1],[1],[2]], ["c"])
df.select(bit_count("c")).show()
# Returns Column | the number of bits that are set in the argument expr as
# an unsigned 64-bit integer, or NULL if the argument is NULL.

+------------+
|bit_count(c)|
+------------+
|           1|
|           1|
|           1|
+------------+



#### pyspark.sql.functions.bit_get(col: ColumnOrName, pos: ColumnOrName) → pyspark.sql.column.Column
Returns the value of the bit (0 or 1) at the specified position. The positions are numbered from right to left, starting at zero. The position argument cannot be negative.

In [180]:
df = spark.createDataFrame([[1],[1],[2],[3],[4],[5],[6],], ["c"])
df.select(bit_get("c", lit(2)).alias("2"), bit_get("c", lit(1)).alias("1"), bit_get("c", lit(0)).alias("0")).show()

+---+---+---+
|  2|  1|  0|
+---+---+---+
|  0|  0|  1|
|  0|  0|  1|
|  0|  1|  0|
|  0|  1|  1|
|  1|  0|  0|
|  1|  0|  1|
|  1|  1|  0|
+---+---+---+



#### pyspark.sql.functions.getbit(col: ColumnOrName, pos: ColumnOrName) → pyspark.sql.column.Column
Returns the value of the bit (0 or 1) at the specified position. The positions are numbered from right to left, starting at zero. The position argument cannot be negative.

In [181]:
import pyspark.sql.functions as sf
spark.createDataFrame(
    [[1], [1], [2]], ["c"]
).select(sf.getbit("c", sf.lit(1))).show()

+------------+
|getbit(c, 1)|
+------------+
|           0|
|           0|
|           1|
+------------+



## [Call Functions](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/functions.html#call-functions)

#### pyspark.sql.functions.call_function(funcName: str, *cols: ColumnOrName) → pyspark.sql.column.Column
Call a SQL function.

`print(call_function.__doc__)`

    Call a SQL function.

    .. versionadded:: 3.5.0

    Parameters
    ----------
    funcName : str
        function name that follows the SQL identifier syntax (can be quoted, can be qualified)
    cols : :class:`~pyspark.sql.Column` or str
        column names or :class:`~pyspark.sql.Column`\s to be used in the function

    Returns
    -------
    :class:`~pyspark.sql.Column`
        result of executed function.

    Examples
    --------
    >>> from pyspark.sql.functions import call_udf, col
    >>> from pyspark.sql.types import IntegerType, StringType
    >>> df = spark.createDataFrame([(1, "a"),(2, "b"), (3, "c")],["id", "name"])
    >>> _ = spark.udf.register("intX2", lambda i: i * 2, IntegerType())
    >>> df.select(call_function("intX2", "id")).show()
    +---------+
    |intX2(id)|
    +---------+
    |        2|
    |        4|
    |        6|
    +---------+
    >>> _ = spark.udf.register("strX2", lambda s: s * 2, StringType())
    >>> df.select(call_function("strX2", col("name"))).show()
    +-----------+
    |strX2(name)|
    +-----------+
    |         aa|
    |         bb|
    |         cc|
    +-----------+
    >>> df.select(call_function("avg", col("id"))).show()
    +-------+
    |avg(id)|
    +-------+
    |    2.0|
    +-------+
    >>> _ = spark.sql("CREATE FUNCTION custom_avg AS 'test.org.apache.spark.sql.MyDoubleAvg'")
    ... # doctest: +SKIP
    >>> df.select(call_function("custom_avg", col("id"))).show()
    ... # doctest: +SKIP
    +------------------------------------+
    |spark_catalog.default.custom_avg(id)|
    +------------------------------------+
    |                               102.0|
    +------------------------------------+
    >>> df.select(call_function("spark_catalog.default.custom_avg", col("id"))).show()
    ... # doctest: +SKIP
    +------------------------------------+
    |spark_catalog.default.custom_avg(id)|
    +------------------------------------+
    |                               102.0|
    +------------------------------------+
    

In [3]:
df = spark.createDataFrame([(1, "a"),(2, "b"), (3, "c")],["id", "name"])
_ = spark.udf.register("intX2", lambda i: i * 2, IntegerType())
df.select(call_function("intX2", "id")).show()

+---------+
|intX2(id)|
+---------+
|        2|
|        4|
|        6|
+---------+



In [4]:
_ = spark.udf.register("strX2", lambda s: s * 2, StringType())
df.select(call_function("strX2", col("name"))).show()

+-----------+
|strX2(name)|
+-----------+
|         aa|
|         bb|
|         cc|
+-----------+



In [5]:
df.select(call_function("avg", col("id"))).show()

+-------+
|avg(id)|
+-------+
|    2.0|
+-------+



In [6]:
# _ = spark.sql("CREATE FUNCTION custom_avg AS 'test.org.apache.spark.sql.MyDoubleAvg'")
# df.select(call_function("custom_avg", col("id"))).show()
# df.select(call_function("spark_catalog.default.custom_avg", col("id"))).show()

#### pyspark.sql.functions.call_udf(udfName: str, *cols: ColumnOrName) → pyspark.sql.column.Column
Call an user-defined function.

`print(call_udf.__doc__)`

    Call an user-defined function.

    .. versionadded:: 3.4.0

    Parameters
    ----------
    udfName : str
        name of the user defined function (UDF)
    cols : :class:`~pyspark.sql.Column` or str
        column names or :class:`~pyspark.sql.Column`\s to be used in the UDF

    Returns
    -------
    :class:`~pyspark.sql.Column`
        result of executed udf.

    Examples
    --------
    >>> from pyspark.sql.functions import call_udf, col
    >>> from pyspark.sql.types import IntegerType, StringType
    >>> df = spark.createDataFrame([(1, "a"),(2, "b"), (3, "c")],["id", "name"])
    >>> _ = spark.udf.register("intX2", lambda i: i * 2, IntegerType())
    >>> df.select(call_udf("intX2", "id")).show()
    +---------+
    |intX2(id)|
    +---------+
    |        2|
    |        4|
    |        6|
    +---------+
    >>> _ = spark.udf.register("strX2", lambda s: s * 2, StringType())
    >>> df.select(call_udf("strX2", col("name"))).show()
    +-----------+
    |strX2(name)|
    +-----------+
    |         aa|
    |         bb|
    |         cc|
    +-----------+

In [95]:
_ = spark.udf.register("intX1.5", lambda i: i * 1.5, IntegerType())
students.limit(10).select(call_udf("intX2", "math score")).show()

+-----------------+
|intX2(math score)|
+-----------------+
|              144|
|              138|
|              180|
|               94|
|              152|
|              142|
|              176|
|               80|
|              128|
|               76|
+-----------------+



In [101]:
_ = spark.udf.register("strX2", lambda s: s * 2, StringType())
tour.limit(5).select(call_udf("strX2", col("Player name"))).show(truncate=False)

23/10/22 16:03:15 WARN SimpleFunctionRegistry: The function strx2 replaced a previously registered function.


+------------------------------+
|strX2(Player name)            |
+------------------------------+
|Robert GarrigusRobert Garrigus|
|Bubba WatsonBubba Watson      |
|Dustin JohnsonDustin Johnson  |
|Brett WetterichBrett Wetterich|
|J.B. HolmesJ.B. Holmes        |
+------------------------------+



#### pyspark.sql.functions.pandas_udf(f=None, returnType=None, functionType=None)
Creates a pandas user defined function (a.k.a. vectorized user defined function).

Pandas UDFs are user defined functions that are executed by Spark using Arrow to transfer data and Pandas to work with the data, which allows vectorized operations. A Pandas UDF is defined using the pandas_udf as a decorator or to wrap the function, and no additional configuration is required. A Pandas UDF behaves as a regular PySpark function API in general.

Parameters:
- f: function, optional |
user-defined function. A python function if used as a standalone function
- return: Typepyspark.sql.types.DataType or str, optional | 
the return type of the user-defined function. The value can be either a pyspark.sql.types.DataType object or a DDL-formatted type string.
- function: Typeint, optional | 
an enum value in pyspark.sql.functions.PandasUDFType. Default: SCALAR. This parameter exists for compatibility. Using Python type hints is encouraged.

[why] Get same errors with all code snippets. Probably unmatching versions of libraries

In [125]:
"""
import pandas as pd

@pandas_udf(IntegerType())
def slen(s: pd.Series) -> pd.Series:
    return s.str.len()

df = tour.select("Player name")
df.withColumn("Name_Length", slen(df["Player name"])).show()
"""

'\nimport pandas as pd\n\n@pandas_udf(IntegerType())\ndef slen(s: pd.Series) -> pd.Series:\n    return s.str.len()\n\ndf = tour.select("Player name")\ndf.withColumn("Name_Length", slen(df["Player name"])).show()\n'

In [126]:
from pyspark.sql.functions import PandasUDFType
from pyspark.sql.types import IntegerType
@pandas_udf(IntegerType(), PandasUDFType.SCALAR)
def slen(s):
    return s.str.len()

In [127]:
@pandas_udf("col1 string, col2 long")
def func(s1: pd.Series, s2: pd.Series, s3: pd.DataFrame) -> pd.DataFrame:
    s3['col2'] = s1 + s2.str.len()
    return s3

# Create a Spark DataFrame that has three columns including a struct column.
df = spark.createDataFrame(
    [[1, "a string", ("a nested string",)]],
    "long_col long, string_col string, struct_col struct<col1:string>")
df.printSchema()

root
 |-- long_col: long (nullable = true)
 |-- string_col: string (nullable = true)
 |-- struct_col: struct (nullable = true)
 |    |-- col1: string (nullable = true)



In [128]:
df.select(func("long_col", "string_col", "struct_col")).printSchema()

root
 |-- func(long_col, string_col, struct_col): struct (nullable = true)
 |    |-- col1: string (nullable = true)
 |    |-- col2: long (nullable = true)



In [130]:
"""
@pandas_udf("string")
def to_upper(s: pd.Series) -> pd.Series:
    return s.str.upper()

df = spark.createDataFrame([("John Doe",)], ("name",))
df.select(to_upper("name")).show()
"""

'\n@pandas_udf("string")\ndef to_upper(s: pd.Series) -> pd.Series:\n    return s.str.upper()\n\ndf = spark.createDataFrame([("John Doe",)], ("name",))\ndf.select(to_upper("name")).show()\n'

In [132]:
"""
@pandas_udf("first string, last string")
def split_expand(s: pd.Series) -> pd.DataFrame:
    return s.str.split(expand=True)

df = spark.createDataFrame([("John Doe",)], ("name",))
df.select(split_expand("name")).show()
"""

'\n@pandas_udf("first string, last string")\ndef split_expand(s: pd.Series) -> pd.DataFrame:\n    return s.str.split(expand=True)\n\ndf = spark.createDataFrame([("John Doe",)], ("name",))\ndf.select(split_expand("name")).show()\n'

In [136]:
"""
from typing import Iterator

@pandas_udf("long")
def calculate(iterator: Iterator[pd.Series]) -> Iterator[pd.Series]:
    # Do some expensive initialization with a state
    state = very_expensive_initialization()
    for x in iterator:
        # Use that state for whole iterator.
        yield calculate_with_state(x, state)

df.select(calculate("value")).show()
"""

'\nfrom typing import Iterator\n\n@pandas_udf("long")\ndef calculate(iterator: Iterator[pd.Series]) -> Iterator[pd.Series]:\n    # Do some expensive initialization with a state\n    state = very_expensive_initialization()\n    for x in iterator:\n        # Use that state for whole iterator.\n        yield calculate_with_state(x, state)\n\ndf.select(calculate("value")).show()\n'

In [138]:
"""
from typing import Iterator
@pandas_udf("long")
def plus_one(iterator: Iterator[pd.Series]) -> Iterator[pd.Series]:
    for s in iterator:
        yield s + 1

df = spark.createDataFrame(pd.DataFrame([1, 2, 3], columns=["v"]))
df.select(plus_one(df.v)).show()
"""

'\nfrom typing import Iterator\n@pandas_udf("long")\ndef plus_one(iterator: Iterator[pd.Series]) -> Iterator[pd.Series]:\n    for s in iterator:\n        yield s + 1\n\ndf = spark.createDataFrame(pd.DataFrame([1, 2, 3], columns=["v"]))\ndf.select(plus_one(df.v)).show()\n'

In [140]:
"""
from typing import Iterator, Tuple
from pyspark.sql.functions import struct, col
@pandas_udf("long")
def multiply(iterator: Iterator[Tuple[pd.Series, pd.DataFrame]]) -> Iterator[pd.Series]:
    for s1, df in iterator:
        yield s1 * df.v

df = spark.createDataFrame(pd.DataFrame([1, 2, 3], columns=["v"]))
df.withColumn('output', multiply(col("v"), struct(col("v")))).show()
"""

'\nfrom typing import Iterator, Tuple\nfrom pyspark.sql.functions import struct, col\n@pandas_udf("long")\ndef multiply(iterator: Iterator[Tuple[pd.Series, pd.DataFrame]]) -> Iterator[pd.Series]:\n    for s1, df in iterator:\n        yield s1 * df.v\n\ndf = spark.createDataFrame(pd.DataFrame([1, 2, 3], columns=["v"]))\ndf.withColumn(\'output\', multiply(col("v"), struct(col("v")))).show()\n'

In [142]:
"""
@pandas_udf("double")
def mean_udf(v: pd.Series) -> float:
    return v.mean()

df = spark.createDataFrame([(1, 1.0), (1, 2.0), (2, 3.0), (2, 5.0), (2, 10.0)], ("id", "v"))
df.groupby("id").agg(mean_udf(df['v'])).show()
"""

'\n@pandas_udf("double")\ndef mean_udf(v: pd.Series) -> float:\n    return v.mean()\n\ndf = spark.createDataFrame([(1, 1.0), (1, 2.0), (2, 3.0), (2, 5.0), (2, 10.0)], ("id", "v"))\ndf.groupby("id").agg(mean_udf(df[\'v\'])).show()\n'

In [144]:
"""
from pyspark.sql import Window
@pandas_udf("double")
def mean_udf(v: pd.Series) -> float:
    return v.mean()

df = spark.createDataFrame(
    [(1, 1.0), (1, 2.0), (2, 3.0), (2, 5.0), (2, 10.0)], ("id", "v"))
w = Window.partitionBy('id').orderBy('v').rowsBetween(-1, 0)
df.withColumn('mean_v', mean_udf("v").over(w)).show()
"""

'\nfrom pyspark.sql import Window\n@pandas_udf("double")\ndef mean_udf(v: pd.Series) -> float:\n    return v.mean()\n\ndf = spark.createDataFrame(\n    [(1, 1.0), (1, 2.0), (2, 3.0), (2, 5.0), (2, 10.0)], ("id", "v"))\nw = Window.partitionBy(\'id\').orderBy(\'v\').rowsBetween(-1, 0)\ndf.withColumn(\'mean_v\', mean_udf("v").over(w)).show()\n'

In [146]:
print(pandas_udf.__doc__)


    Creates a pandas user defined function (a.k.a. vectorized user defined function).

    Pandas UDFs are user defined functions that are executed by Spark using Arrow to transfer
    data and Pandas to work with the data, which allows vectorized operations. A Pandas UDF
    is defined using the `pandas_udf` as a decorator or to wrap the function, and no
    additional configuration is required. A Pandas UDF behaves as a regular PySpark function
    API in general.

    .. versionadded:: 2.3.0

    .. versionchanged:: 3.4.0
        Supports Spark Connect.

    Parameters
    ----------
    f : function, optional
        user-defined function. A python function if used as a standalone function
    returnType : :class:`pyspark.sql.types.DataType` or str, optional
        the return type of the user-defined function. The value can be either a
        :class:`pyspark.sql.types.DataType` object or a DDL-formatted type string.
    functionType : int, optional
        an enum value in :clas

#### pyspark.sql.functions.udf(f: Union[Callable[[…], Any], DataTypeOrString, None] = None, returnType: DataTypeOrString = StringType(), *, useArrow: Optional[bool] = None) → Union[UserDefinedFunctionLike, Callable[[Callable[[…], Any]], UserDefinedFunctionLike]]
Creates a user defined function (UDF).

Parameters: 
- f: function |
python function if used as a standalone function
- return: Typepyspark.sql.types.DataType or str | 
the return type of the user-defined function. The value can be either a pyspark.sql.types.DataType object or a DDL-formatted type string.
- use: Arrowbool or None | 
whether to use Arrow to optimize the (de)serialization. When it is None, the Spark config “spark.sql.execution.pythonUDF.arrow.enabled” takes effect.

In [147]:
import random
random_udf = udf(lambda: int(random.random() * 100), IntegerType()).asNondeterministic()

In [149]:
students.limit(5).withColumn("Random int", random_udf()).show(5)

+------+--------------+---------------------------+------------+-----------------------+----------+-------------+-------------+----------+
|gender|race/ethnicity|parental level of education|       lunch|test preparation course|math score|reading score|writing score|Random int|
+------+--------------+---------------------------+------------+-----------------------+----------+-------------+-------------+----------+
|female|       group B|          bachelor's degree|    standard|                   none|        72|           72|           74|         2|
|female|       group C|               some college|    standard|              completed|        69|           90|           88|        69|
|female|       group B|            master's degree|    standard|                   none|        90|           95|           93|        38|
|  male|       group A|         associate's degree|free/reduced|                   none|        47|           57|           44|         3|
|  male|       group C|    

In [148]:
from pyspark.sql.types import IntegerType
slen = udf(lambda s: len(s), IntegerType())
@udf
def to_upper(s):
    if s is not None:
        return s.upper()

@udf(returnType=IntegerType())
def add_one(x):
    if x is not None:
        return x + 1

df = spark.createDataFrame([(1, "John Doe", 21)], ("id", "name", "age"))
df.select(slen("name").alias("slen(name)"), to_upper("name"), add_one("age")).show()

+----------+--------------+------------+
|slen(name)|to_upper(name)|add_one(age)|
+----------+--------------+------------+
|         8|      JOHN DOE|          22|
+----------+--------------+------------+



#### pyspark.sql.functions.udtf(cls: Optional[Type] = None, *, returnType: Union[pyspark.sql.types.StructType, str], useArrow: Optional[bool] = None) → Union[pyspark.sql.udtf.UserDefinedTableFunction, Callable[[Type], pyspark.sql.udtf.UserDefinedTableFunction]]
Creates a user defined table function (UDTF).

Parameters:
- cls: class | 
the Python user-defined table function handler class.
- return: Typepyspark.sql.types.StructType or str | 
the return type of the user-defined table function. The value can be either a pyspark.sql.types.StructType object or a DDL-formatted struct type string.
- use: Arrowbool or None, optional | 
whether to use Arrow to optimize the (de)serializations. When it’s set to None, the Spark config “spark.sql.execution.pythonUDTF.arrow.enabled” is used.

In [166]:
class PlusOne:
    def eval(self, a: int):
        yield a + 1,
plus_one = udtf(PlusOne, returnType="r: int").asDeterministic()

In [167]:
class TestUDTF:
    def eval(self, *args: Any):
        yield "hello", "world"

test_udtf = udtf(TestUDTF, returnType="c1: string, c2: string")
test_udtf().show()

+-----+-----+
|   c1|   c2|
+-----+-----+
|hello|world|
+-----+-----+



In [168]:
@udtf(returnType="c1: int, c2: int")
class PlusOne:
    def eval(self, x: int):
        yield x, x + 1

from pyspark.sql.functions import lit
PlusOne(lit(1)).show()

+---+---+
| c1| c2|
+---+---+
|  1|  2|
+---+---+



In [173]:
@udtf(returnType="c1: int, c2: int", useArrow=True)
class ArrowPlusOne:
    def eval(self, x: int):
        yield x, x + 1

# eval_type = read_int(infile) Error
# ArrowPlusOne(lit(1)).show()

#### pyspark.sql.functions.unwrap_udt(col: ColumnOrName) → pyspark.sql.column.Column
Unwrap UDT data type column into its underlying type.

In [165]:
"""
class MyUDT:
    def __init__(self, value):
        self.value = value

data = [(MyUDT(42),)]
schema = StructType([StructField("my_udt", MyUDTType(), True)])
df = spark.createDataFrame(data, schema)
"""
True

True

In [ ]:
"""
result_df = df.select(unwrap_udt(col("my_udt")).alias("my_udt_value"))
result_df.show()
"""

In [164]:
"""
from pyspark.sql.functions import col
from pyspark.sql.types import UserDefinedType

# Визначаємо користувацький тип даних MyUDT
class MyUDT(UserDefinedType):

    def __init__(self, value = None):
        self.value = value
        
    def simpleString(self):
        return "MyUDT"

    def serialize(self, obj):
        return str(obj).encode('utf-8')

    def deserialize(self, datum):
        return MyUDT(int(datum.decode('utf-8')))

# Оголошуємо функцію unwrap_udt з використанням користувацького типу MyUDT
@pandas_udf(MyUDT())
def unwrap_udt(s: pd.Series) -> pd.Series:
    return s

# Створюємо DataFrame
data = [(MyUDT(42),), (MyUDT(56),)]
df = spark.createDataFrame(data, ["my_udt"])

# Використовуємо unwrap_udt для розгортання користувацького типу MyUDT
result_df = df.select(unwrap_udt(col("my_udt")).alias("my_udt_value"))

# Виводимо результат
result_df.show()
"""
True

True

## [Misc Functions](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/functions.html#misc-functions)

#### pyspark.sql.functions.aes_decrypt(input: ColumnOrName, key: ColumnOrName, mode: Optional[ColumnOrName] = None, padding: Optional[ColumnOrName] = None, aad: Optional[ColumnOrName] = None) → pyspark.sql.column.Column

##### The Advanced Encryption Standard (AES) 
Returns a decrypted value of input using AES in mode with padding. Key lengths of 16, 24 and 32 bits are supported. Supported combinations of (mode, padding) are (‘ECB’, ‘PKCS’), (‘GCM’, ‘NONE’) and (‘CBC’, ‘PKCS’). Optional additional authenticated data (AAD) is only supported for GCM. If provided for encryption, the identical AAD value must be provided for decryption. The default mode is GCM.

Parameters:
- input: Column or str | The binary value to decrypt.
- key: Column or str | The passphrase to use to decrypt the data.
- mode: Column or str, optional | Specifies which block cipher mode should be used to decrypt messages. Valid modes: ECB, GCM, CBC.
- padding: Column or str, optional | Specifies how to pad messages whose length is not a multiple of the block size. Valid values: PKCS, NONE, DEFAULT. The DEFAULT padding means PKCS for ECB, NONE for GCM and PKCS for CBC.
- aad: Column or str, optional | Optional additional authenticated data. Only supported for GCM mode. This can be any free-form input and must be provided for both encryption and decryption.

In [7]:
df = spark.createDataFrame([(
    "AAAAAAAAAAAAAAAAQiYi+sTLm7KD9UcZ2nlRdYDe/PX4",
    "abcdefghijklmnop12345678ABCDEFGH", "GCM", "DEFAULT",
    "This is an AAD mixed into the input",)],
    ["input", "key", "mode", "padding", "aad"]
)
df.select(aes_decrypt(
    unbase64(df.input), df.key, df.mode, df.padding, df.aad).alias('r')
).collect()

[Row(r=bytearray(b'Spark'))]

In [8]:
df = spark.createDataFrame([(
    "AAAAAAAAAAAAAAAAAAAAAPSd4mWyMZ5mhvjiAPQJnfg=",
    "abcdefghijklmnop12345678ABCDEFGH", "CBC", "DEFAULT",)],
    ["input", "key", "mode", "padding"]
)
df.select(aes_decrypt(
    unbase64(df.input), df.key, df.mode, df.padding).alias('r')
).collect()

[Row(r=bytearray(b'Spark'))]

In [9]:
df.select(aes_decrypt(unbase64(df.input), df.key, df.mode).alias('r')).collect()

[Row(r=bytearray(b'Spark'))]

In [10]:
df = spark.createDataFrame([(
    "83F16B2AA704794132802D248E6BFD4E380078182D1544813898AC97E709B28A94",
    "0000111122223333",)],
    ["input", "key"]
)
df.select(aes_decrypt(unhex(df.input), df.key).alias('r')).collect()

[Row(r=bytearray(b'Spark'))]

#### pyspark.sql.functions.aes_encrypt(input: ColumnOrName, key: ColumnOrName, mode: Optional[ColumnOrName] = None, padding: Optional[ColumnOrName] = None, iv: Optional[ColumnOrName] = None, aad: Optional[ColumnOrName] = None) → pyspark.sql.column.Column

##### The Advanced Encryption Standard (AES) 
Returns an encrypted value of input using AES in given mode with the specified padding. Key lengths of 16, 24 and 32 bits are supported. Supported combinations of (mode, padding) are (‘ECB’, ‘PKCS’), (‘GCM’, ‘NONE’) and (‘CBC’, ‘PKCS’). Optional initialization vectors (IVs) are only supported for CBC and GCM modes. These must be 16 bytes for CBC and 12 bytes for GCM. If not provided, a random vector will be generated and prepended to the output. Optional additional authenticated data (AAD) is only supported for GCM. If provided for encryption, the identical AAD value must be provided for decryption. The default mode is GCM.

Parameters: 
- input: Column or str | The binary value to encrypt.
- key: Column or str | The passphrase to use to encrypt the data.
- mode: Column or str, optional | Specifies which block cipher mode should be used to encrypt messages. Valid modes: ECB, GCM, CBC.
- padding: Column or str, optional | Specifies how to pad messages whose length is not a multiple of the block size. Valid values: PKCS, NONE, DEFAULT. The DEFAULT padding means PKCS for ECB, NONE for GCM and PKCS for CBC.
- iv: Column or str, optional | Optional initialization vector. Only supported for CBC and GCM modes. Valid values: None or “”. 16-byte array for CBC mode. 12-byte array for GCM mode.
- aad: Column or str, optional | Optional additional authenticated data. Only supported for GCM mode. This can be any free-form input and must be provided for both encryption and decryption.

In [11]:
df = spark.createDataFrame([(
    "Spark", "abcdefghijklmnop12345678ABCDEFGH", "GCM", "DEFAULT",
    "000000000000000000000000", "This is an AAD mixed into the input",)],
    ["input", "key", "mode", "padding", "iv", "aad"]
)
df.select(base64(aes_encrypt(
    df.input, df.key, df.mode, df.padding, to_binary(df.iv, lit("hex")), df.aad)
).alias('r')).collect()

[Row(r='AAAAAAAAAAAAAAAAQiYi+sTLm7KD9UcZ2nlRdYDe/PX4')]

In [12]:
df.select(base64(aes_encrypt(
    df.input, df.key, df.mode, df.padding, to_binary(df.iv, lit("hex")))
).alias('r')).collect()

[Row(r='AAAAAAAAAAAAAAAAQiYi+sRNYDAOTjdSEcYBFsAWPL1f')]

In [13]:
df = spark.createDataFrame([(
    "Spark SQL", "1234567890abcdef", "ECB", "PKCS",)],
    ["input", "key", "mode", "padding"]
)
df.select(aes_decrypt(aes_encrypt(df.input, df.key, df.mode, df.padding),
    df.key, df.mode, df.padding).alias('r')
).collect()

[Row(r=bytearray(b'Spark SQL'))]

In [14]:
df = spark.createDataFrame([(
    "Spark SQL", "0000111122223333", "ECB",)],
    ["input", "key", "mode"]
)
df.select(aes_decrypt(aes_encrypt(df.input, df.key, df.mode),
    df.key, df.mode).alias('r')
).collect()

[Row(r=bytearray(b'Spark SQL'))]

In [15]:
df = spark.createDataFrame([(
    "Spark SQL", "abcdefghijklmnop",)],
    ["input", "key"]
)
df.select(aes_decrypt(
    unbase64(base64(aes_encrypt(df.input, df.key))), df.key
).cast("STRING").alias('r')).collect()

[Row(r='Spark SQL')]

In [16]:
key_1 = "0000111122223333"
key_2 = "abcdefghijklmnop"

df = spark.createDataFrame([(
    "83F16B2AA704794132802D248E6BFD4E380078182D1544813898AC97E709B28A94",
    key_1,)],
    ["input", "key"]
)
df.select(aes_decrypt(unhex(df.input), df.key).alias('r')).collect()

[Row(r=bytearray(b'Spark'))]

In [17]:
df = spark.createDataFrame([(
    "Spark SQL", key_2,)],
    ["input", "key"]
)

df_key_2_en = df.select(aes_encrypt(df.input, df.key).cast("STRING").alias('r'))
df_key_2_en.collect()

[Row(r='!\x196�:���\x7f�4�]��hT�5\x7fshm<\x0b4wO\x06�\r��\x10�1')]

In [18]:
df_key_2_de = df_key_2_en.select(aes_decrypt("r", lit(key_2)).cast("STRING").alias('r'))
df_key_2_de.collect()

[Row(r='Spark SQL')]

#### pyspark.sql.functions.bitmap_bit_position(col: ColumnOrName) → pyspark.sql.column.Column
Returns the bit position for the given input column.

In [19]:
df = spark.createDataFrame([(123,)], ["a"])
df.select(bitmap_bit_position(df.a).alias("r")).collect()

[Row(r=122)]

In [20]:
tour.limit(20).select("Value", bitmap_bit_position("Value")).collect()

[Row(Value='71', bitmap_bit_position(Value)=70),
 Row(Value='77', bitmap_bit_position(Value)=76),
 Row(Value='83', bitmap_bit_position(Value)=82),
 Row(Value='54', bitmap_bit_position(Value)=53),
 Row(Value='100', bitmap_bit_position(Value)=99),
 Row(Value='63', bitmap_bit_position(Value)=62),
 Row(Value='88', bitmap_bit_position(Value)=87),
 Row(Value='64', bitmap_bit_position(Value)=63),
 Row(Value='64', bitmap_bit_position(Value)=63),
 Row(Value='92', bitmap_bit_position(Value)=91),
 Row(Value='75', bitmap_bit_position(Value)=74),
 Row(Value='54', bitmap_bit_position(Value)=53),
 Row(Value='76', bitmap_bit_position(Value)=75),
 Row(Value='94', bitmap_bit_position(Value)=93),
 Row(Value='82', bitmap_bit_position(Value)=81),
 Row(Value='85', bitmap_bit_position(Value)=84),
 Row(Value='79', bitmap_bit_position(Value)=78),
 Row(Value='89', bitmap_bit_position(Value)=88),
 Row(Value='88', bitmap_bit_position(Value)=87),
 Row(Value='91', bitmap_bit_position(Value)=90)]

In [21]:
tour.limit(20).select(bitmap_bit_position(tour.Value)).collect()

[Row(bitmap_bit_position(Value)=70),
 Row(bitmap_bit_position(Value)=76),
 Row(bitmap_bit_position(Value)=82),
 Row(bitmap_bit_position(Value)=53),
 Row(bitmap_bit_position(Value)=99),
 Row(bitmap_bit_position(Value)=62),
 Row(bitmap_bit_position(Value)=87),
 Row(bitmap_bit_position(Value)=63),
 Row(bitmap_bit_position(Value)=63),
 Row(bitmap_bit_position(Value)=91),
 Row(bitmap_bit_position(Value)=74),
 Row(bitmap_bit_position(Value)=53),
 Row(bitmap_bit_position(Value)=75),
 Row(bitmap_bit_position(Value)=93),
 Row(bitmap_bit_position(Value)=81),
 Row(bitmap_bit_position(Value)=84),
 Row(bitmap_bit_position(Value)=78),
 Row(bitmap_bit_position(Value)=88),
 Row(bitmap_bit_position(Value)=87),
 Row(bitmap_bit_position(Value)=90)]

#### pyspark.sql.functions.bitmap_bucket_number(col: ColumnOrName) → pyspark.sql.column.Column
Returns the bucket number for the given input column.

In [22]:
df = spark.createDataFrame([(123,)], ["a"])
df.select(bitmap_bucket_number(df.a).alias("r")).collect()

23/10/22 15:48:32 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


[Row(r=1)]

In [23]:
tour.limit(20).select("Value", bitmap_bucket_number("Value")).collect()

[Row(Value='71', bitmap_bucket_number(Value)=1),
 Row(Value='77', bitmap_bucket_number(Value)=1),
 Row(Value='83', bitmap_bucket_number(Value)=1),
 Row(Value='54', bitmap_bucket_number(Value)=1),
 Row(Value='100', bitmap_bucket_number(Value)=1),
 Row(Value='63', bitmap_bucket_number(Value)=1),
 Row(Value='88', bitmap_bucket_number(Value)=1),
 Row(Value='64', bitmap_bucket_number(Value)=1),
 Row(Value='64', bitmap_bucket_number(Value)=1),
 Row(Value='92', bitmap_bucket_number(Value)=1),
 Row(Value='75', bitmap_bucket_number(Value)=1),
 Row(Value='54', bitmap_bucket_number(Value)=1),
 Row(Value='76', bitmap_bucket_number(Value)=1),
 Row(Value='94', bitmap_bucket_number(Value)=1),
 Row(Value='82', bitmap_bucket_number(Value)=1),
 Row(Value='85', bitmap_bucket_number(Value)=1),
 Row(Value='79', bitmap_bucket_number(Value)=1),
 Row(Value='89', bitmap_bucket_number(Value)=1),
 Row(Value='88', bitmap_bucket_number(Value)=1),
 Row(Value='91', bitmap_bucket_number(Value)=1)]

#### pyspark.sql.functions.bitmap_construct_agg(col: ColumnOrName) → pyspark.sql.column.Column
Returns a bitmap with the positions of the bits set from all the values from the input column. The input column will most likely be bitmap_bit_position().

In [24]:
df = spark.createDataFrame([(1,),(2,),(3,)], ["a"])
df.select(substring(hex(
    bitmap_construct_agg(bitmap_bit_position(df.a))
), 0, 6).alias("r")).collect()

[Row(r='070000')]

In [25]:
students.limit(10).select("math score", bitmap_bit_position("math score")).collect()

[Row(math score=72, bitmap_bit_position(math score)=71),
 Row(math score=69, bitmap_bit_position(math score)=68),
 Row(math score=90, bitmap_bit_position(math score)=89),
 Row(math score=47, bitmap_bit_position(math score)=46),
 Row(math score=76, bitmap_bit_position(math score)=75),
 Row(math score=71, bitmap_bit_position(math score)=70),
 Row(math score=88, bitmap_bit_position(math score)=87),
 Row(math score=40, bitmap_bit_position(math score)=39),
 Row(math score=64, bitmap_bit_position(math score)=63),
 Row(math score=38, bitmap_bit_position(math score)=37)]

In [26]:
students.select(substring(hex(
    bitmap_construct_agg(bitmap_bit_position("math score"))
), 0, 6).alias("r")).collect()

[Row(r='8100E6')]

In [27]:
students.select(hex(
    bitmap_construct_agg(bitmap_bit_position("math score"))
).alias("r")).collect()

[Row(r='8100E6BEFFFFFFFFFFFFFFFF0F000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

In [28]:
students.select(bitmap_construct_agg(bitmap_bit_position("math score")).alias("r")).collect()

[Row(r=bytearray(b'\x81\x00\xe6\xbe\xff\xff\xff\xff\xff\xff\xff\xff\x0f\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\

In [29]:
students.limit(20).select(substring(hex(
    bitmap_construct_agg(bitmap_bit_position("math score"))
), 0, 6).alias("r")).collect()

[Row(r='000002')]

In [30]:
students.limit(20).select(hex(bitmap_construct_agg(bitmap_bit_position("math score"))).alias("r")).collect()

[Row(r='00000200A0602282D128800200000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

#### pyspark.sql.functions.bitmap_count(col: ColumnOrName) → pyspark.sql.column.Column
Returns the number of set bits in the input bitmap.

In [31]:
df = spark.createDataFrame([("FFFF",)], ["a"])
df.select(bitmap_count(to_binary(df.a, lit("hex"))).alias('r')).collect()

[Row(r=16)]

In [32]:
students.limit(10).select(bitmap_count(to_binary("math score", lit("hex"))).alias("r")).collect()

[Row(r=4),
 Row(r=4),
 Row(r=2),
 Row(r=4),
 Row(r=5),
 Row(r=4),
 Row(r=2),
 Row(r=1),
 Row(r=3),
 Row(r=3)]

#### pyspark.sql.functions.bitmap_or_agg(col: ColumnOrName) → pyspark.sql.column.Column
Returns a bitmap that is the bitwise OR of all of the bitmaps from the input column. The input column should be bitmaps created from bitmap_construct_agg().

In [33]:
df = spark.createDataFrame([("10",),("20",),("40",)], ["a"])
df.select(substring(hex(
    bitmap_or_agg(to_binary(df.a, lit("hex")))
), 0, 6).alias("r")).collect()

[Row(r='700000')]

In [34]:
students.limit(10).select(substring(hex(
    bitmap_or_agg(to_binary("math score", lit("hex")))), 0, 6).alias("r")).collect()

[Row(r='FF0000')]

#### pyspark.sql.functions.current_catalog() → pyspark.sql.column.Column
Returns the current catalog.

In [35]:
spark.range(1).select(current_catalog()).show()

+-----------------+
|current_catalog()|
+-----------------+
|    spark_catalog|
+-----------------+



#### pyspark.sql.functions.current_database() → pyspark.sql.column.Column[source]
Returns the current database.

In [36]:
spark.range(1).select(current_database()).show()

+------------------+
|current_database()|
+------------------+
|           default|
+------------------+



#### pyspark.sql.functions.current_schema() → pyspark.sql.column.Column
Returns the current database.

In [37]:
spark.range(1).select(current_schema()).show()

+------------------+
|current_database()|
+------------------+
|           default|
+------------------+



#### pyspark.sql.functions.current_user() → pyspark.sql.column.Column

In [38]:
spark.range(1).select(current_user()).show()

+--------------+
|current_user()|
+--------------+
|    zsavchenko|
+--------------+



#### pyspark.sql.functions.input_file_block_length() → pyspark.sql.column.Column
Returns the length of the block being read, or -1 if not available.

In [39]:
df = spark.read.text(path+"students.csv", lineSep=",")
df.select(input_file_block_length().alias('r')).first()

Row(r=72036)

In [40]:
df = spark.read.text(path+"pga_tour_historical.csv", lineSep=",")
df.select(input_file_block_length().alias('r')).first()

Row(r=21305899)

#### pyspark.sql.functions.input_file_block_start() → pyspark.sql.column.Column
Returns the start offset of the block being read, or -1 if not available.

In [41]:
df = spark.read.text(path+"students.csv", lineSep=",")
df.select(input_file_block_start().alias('r')).first()

Row(r=0)

In [42]:
df = spark.read.text(path+"pga_tour_historical.csv", lineSep=",")
df.select(input_file_block_start().alias('r')).first()

Row(r=0)

#### pyspark.sql.functions.md5(col: ColumnOrName) → pyspark.sql.column.Column
Calculates the MD5 digest and returns the value as a 32 character hex string.

In [43]:
spark.createDataFrame([('ABC',)], ['a']).select(md5('a').alias('hash')).collect()

[Row(hash='902fbdd2b1df0c4f70b4a5d23525e932')]

#### pyspark.sql.functions.sha(col: ColumnOrName) → pyspark.sql.column.Column
Returns a sha1 hash value as a hex string of the col.

In [44]:
spark.range(1).select(sha(lit("Spark"))).show(1, False)

+----------------------------------------+
|sha(Spark)                              |
+----------------------------------------+
|85f5955f4b27a9a4c2aab6ffe5d7189fc298b92c|
+----------------------------------------+



#### pyspark.sql.functions.sha1(col: ColumnOrName) → pyspark.sql.column.Column
Returns the hex string result of SHA-1.

In [45]:
spark.createDataFrame([('ABC',)], ['a']).select(sha1('a').alias('hash')).collect()

[Row(hash='3c01bdbb26f358bab27f267924aa2c9a03fcfdb8')]

#### pyspark.sql.functions.sha2(col: ColumnOrName, numBits: int) → pyspark.sql.column.Column
Returns the hex string result of SHA-2 family of hash functions (SHA-224, SHA-256, SHA-384, and SHA-512). The numBits indicates the desired bit length of the result, which must have a value of 224, 256, 384, 512, or 0 (which is equivalent to 256).

Parameters: 
- col: Column or str | target column to compute on.
- num: Bitsint | the desired bit length of the result, which must have a value of 224, 256, 384, 512, or 0 (which is equivalent to 256).

Returns – Column | the column for computed results.

In [46]:
df = spark.createDataFrame([["Alice"], ["Bob"]], ["name"])
df.withColumn("sha2", sha2(df.name, 256)).show(truncate=False)

+-----+----------------------------------------------------------------+
|name |sha2                                                            |
+-----+----------------------------------------------------------------+
|Alice|3bc51062973c458d5a6f2d8d64a023246354ad7e064b1e4e009ec8a0699a3043|
|Bob  |cd9fb1e148ccd8442e5aa74904cc73bf6fb54d1d54d333bd596aa9bb4bb4e961|
+-----+----------------------------------------------------------------+



#### pyspark.sql.functions.crc32(col: ColumnOrName) → pyspark.sql.column.Column
Calculates the cyclic redundancy check value (CRC32) of a binary column and returns the value as a bigint.

In [47]:
spark.createDataFrame([('ABC',)], ['a']).select(crc32('a').alias('crc32')).collect()

[Row(crc32=2743272264)]

#### pyspark.sql.functions.hash(*cols: ColumnOrName) → pyspark.sql.column.Column
Calculates the hash code of given columns, and returns the result as an int column.

In [48]:
df = spark.createDataFrame([('ABC', 'DEF')], ['c1', 'c2'])

df.select(hash('c1').alias('hash')).show()

df.select(hash('c1', 'c2').alias('hash')).show()

+----------+
|      hash|
+----------+
|-757602832|
+----------+

+---------+
|     hash|
+---------+
|599895104|
+---------+



#### pyspark.sql.functions.xxhash64(*cols: ColumnOrName) → pyspark.sql.column.Column
Calculates the hash code of given columns using the 64-bit variant of the xxHash algorithm, and returns the result as a long column. The hash computation uses an initial seed of 42.

In [49]:
df = spark.createDataFrame([('ABC', 'DEF')], ['c1', 'c2'])

df.select(xxhash64('c1').alias('hash')).show()

df.select(xxhash64('c1', 'c2').alias('hash')).show()

+-------------------+
|               hash|
+-------------------+
|4105715581806190027|
+-------------------+

+-------------------+
|               hash|
+-------------------+
|3233247871021311208|
+-------------------+



#### pyspark.sql.functions.assert_true(col: ColumnOrName, errMsg: Union[pyspark.sql.column.Column, str, None] = None) → pyspark.sql.column.Column
Returns null if the input column is true; throws an exception with the provided error message otherwise.

Parameters: 
- col: Column or str | column name or column that represents the input column to test.
- errMsg: Column or str, optional | A Python string literal or column containing the error message.

Returns – Column | null if the input column is true otherwise throws an error with specified message.

In [50]:
df = spark.createDataFrame([(0,1)], ['a', 'b'])
df.select(assert_true(df.a < df.b).alias('r')).collect()

[Row(r=None)]

In [51]:
df.select(assert_true(df.a < df.b, df.a).alias('r')).collect()

[Row(r=None)]

In [52]:
df.select(assert_true(df.a < df.b, 'error').alias('r')).collect()

[Row(r=None)]

In [53]:
from py4j.protocol import Py4JJavaError

try:
    df.select(assert_true(df.a > df.b, 'My error msg').alias('r')).collect()
except Py4JJavaError as e:
    print("My error msg" in e.__str__())

True


23/10/22 15:48:38 ERROR Executor: Exception in task 11.0 in stage 91.0 (TID 496)
java.lang.RuntimeException: My error msg
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenEvaluatorFactory$WholeStageCodegenPartitionEvaluator$$anon$1.hasNext(WholeStageCodegenEvaluatorFactory.scala:43)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$getByteArrayRdd$1(SparkPlan.scala:388)
	at org.apache.spark.rdd.RDD.$anonfun$mapPartitionsInternal$2(RDD.scala:890)
	at org.apache.spark.rdd.RDD.$anonfun$mapPartitionsInternal$2$adapted(RDD.scala:890)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:52)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:364)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:328)
	at org.apache.spark.scheduler.ResultTask.run

#### pyspark.sql.functions.raise_error(errMsg: Union[pyspark.sql.column.Column, str]) → pyspark.sql.column.Column
Throws an exception with the provided error message.

In [54]:
from py4j.protocol import Py4JJavaError

df = spark.range(1)

try:
    df.select(raise_error("My error msg")).show()
except Py4JJavaError as e:
    print("My error msg" in e.__str__())

True


23/10/22 15:48:38 ERROR Executor: Exception in task 11.0 in stage 92.0 (TID 508)
java.lang.RuntimeException: My error msg
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenEvaluatorFactory$WholeStageCodegenPartitionEvaluator$$anon$1.hasNext(WholeStageCodegenEvaluatorFactory.scala:43)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$getByteArrayRdd$1(SparkPlan.scala:388)
	at org.apache.spark.rdd.RDD.$anonfun$mapPartitionsInternal$2(RDD.scala:890)
	at org.apache.spark.rdd.RDD.$anonfun$mapPartitionsInternal$2$adapted(RDD.scala:890)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:52)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:364)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:328)
	at org.apache.spark.scheduler.ResultTask.run

#### pyspark.sql.functions.reflect(*cols: ColumnOrName) → pyspark.sql.column.Column
Calls a method with reflection.

Parameters – cols: Column or str | the first element should be a literal string for the class name, and the second element should be a literal string for the method name, and the remaining are input arguments to the Java method.

In [55]:
df = spark.createDataFrame([("a5cf6c42-0c85-418f-af6c-3e4e5b1328f2",)], ["a"])
df.select(
    reflect(lit("java.util.UUID"), lit("fromString"), df.a).alias('r')
).collect()

spark.scheduler.ResultTask.runTask(ResultTask.scala:93)
	at org.apache.spark.TaskContext.runTaskWithListeners(TaskContext.scala:161)
	at org.apache.spark.scheduler.Task.run(Task.scala:141)
	at org.apache.spark.executor.Executor$TaskRunner.$anonfun$run$4(Executor.scala:620)
	at org.apache.spark.util.SparkErrorUtils.tryWithSafeFinally(SparkErrorUtils.scala:64)
	at org.apache.spark.util.SparkErrorUtils.tryWithSafeFinally$(SparkErrorUtils.scala:61)
	at org.apache.spark.util.Utils$.tryWithSafeFinally(Utils.scala:94)
	at org.apache.spark.executor.Executor$TaskRunner.run(Executor.scala:623)
	at java.base/java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1144)
	at java.base/java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:642)
	at java.base/java.lang.Thread.run(Thread.java:1583)

23/10/22 15:48:38 ERROR TaskSetManager: Task 11 in stage 92.0 failed 1 times; aborting job


[Row(r='a5cf6c42-0c85-418f-af6c-3e4e5b1328f2')]

#### pyspark.sql.functions.hll_sketch_estimate(col: ColumnOrName) → pyspark.sql.column.Column
Returns the estimated number of unique values given the binary representation of a Datasketches HllSketch.

[why] why not use hll_sketch_estimate all time?

Datasketches HyperLogLog (HLL) Sketch - це структура даних, призначена для наближеного підрахунку унікальних елементів у великих наборах даних. Вона використовує алгоритм HyperLogLog для оцінки кількості різних елементів у масиві, не зберігаючи всі ці елементи, а лише важливі характеристики.

In [56]:
df = spark.createDataFrame([1,2,2,3], "INT")
df = df.agg(hll_sketch_estimate(hll_sketch_agg("value")).alias("distinct_cnt"))
df.show()

+------------+
|distinct_cnt|
+------------+
|           3|
+------------+



In [57]:
from time import time
start = time()
tour.agg(hll_sketch_estimate(hll_sketch_agg("Value")).alias("distinct_cnt")).show()
print(f"execution_time - {time() - start}")

+------------+
|distinct_cnt|
+------------+
|      179970|
+------------+

execution_time - 1.1872360706329346


In [58]:
start = time()
print(tour.select("Value").distinct().count())
print(f"execution_time - {time() - start}")

180981
execution_time - 1.5197079181671143


In [59]:
start = time()
students.agg(hll_sketch_estimate(hll_sketch_agg("math score")).alias("distinct_cnt")).show()
print(f"execution_time - {time() - start}")

+------------+
|distinct_cnt|
+------------+
|          81|
+------------+

execution_time - 0.07657694816589355


In [60]:
start = time()
print(students.select("math score").distinct().count())
print(f"execution_time - {time() - start}")

81
execution_time - 0.23991703987121582


#### pyspark.sql.functions.hll_union(col1: ColumnOrName, col2: ColumnOrName, allowDifferentLgConfigK: Optional[bool] = None) → pyspark.sql.column.Column
Merges two binary representations of Datasketches HllSketch objects, using a Datasketches Union object. Throws an exception if sketches have different lgConfigK values and allowDifferentLgConfigK is unset or set to false.

In [61]:
df = spark.createDataFrame([(1,4),(2,5),(2,5),(3,6)], "struct<v1:int,v2:int>")
df = df.agg(hll_sketch_agg("v1").alias("sketch1"), hll_sketch_agg("v2").alias("sketch2"))
df = df.withColumn("distinct_cnt", hll_sketch_estimate(hll_union("sketch1", "sketch2")))
df.show()
df.drop("sketch1", "sketch2").show()

+--------------------+--------------------+------------+
|             sketch1|             sketch2|distinct_cnt|
+--------------------+--------------------+------------+
|[02 01 07 0C 03 0...|[02 01 07 0C 03 0...|           6|
+--------------------+--------------------+------------+

+------------+
|distinct_cnt|
+------------+
|           6|
+------------+



In [62]:
df = students.agg(hll_sketch_agg("math score").alias("sketch1"),
                  hll_sketch_agg("reading score").alias("sketch2"),
                  hll_sketch_agg("writing score").alias("sketch3"),)
df = (df.withColumn("distinct_cnt1", hll_sketch_estimate(hll_union("sketch1", "sketch2")))
        .withColumn("distinct_cnt2", hll_sketch_estimate(hll_union("sketch1", "sketch3")))
        .withColumn("distinct_cnt3", hll_sketch_estimate(hll_union("sketch2", "sketch3")))
        .withColumn("distinct_cnt4", hll_sketch_estimate(hll_union("sketch3", "sketch2")))
)
df.show()
df.drop("sketch1", "sketch2", "sketch3").show()

+--------------------+--------------------+--------------------+-------------+-------------+-------------+-------------+
|             sketch1|             sketch2|             sketch3|distinct_cnt1|distinct_cnt2|distinct_cnt3|distinct_cnt4|
+--------------------+--------------------+--------------------+-------------+-------------+-------------+-------------+
|[03 01 07 0C 07 0...|[03 01 07 0C 07 0...|[03 01 07 0C 07 0...|           83|           83|           82|           82|
+--------------------+--------------------+--------------------+-------------+-------------+-------------+-------------+

+-------------+-------------+-------------+-------------+
|distinct_cnt1|distinct_cnt2|distinct_cnt3|distinct_cnt4|
+-------------+-------------+-------------+-------------+
|           83|           83|           82|           82|
+-------------+-------------+-------------+-------------+



#### pyspark.sql.functions.java_method(*cols: ColumnOrName) → pyspark.sql.column.Column
Calls a method with reflection.

Parameters – cols: Column or str | the first element should be a literal string for the class name, and the second element should be a literal string for the method name, and the remaining are input arguments to the Java method.

In [63]:
spark.range(1).select(
    java_method(
        lit("java.util.UUID"),
        lit("fromString"),
        lit("a5cf6c42-0c85-418f-af6c-3e4e5b1328f2")
    )
).show(truncate=False)

+-----------------------------------------------------------------------------+
|java_method(java.util.UUID, fromString, a5cf6c42-0c85-418f-af6c-3e4e5b1328f2)|
+-----------------------------------------------------------------------------+
|a5cf6c42-0c85-418f-af6c-3e4e5b1328f2                                         |
+-----------------------------------------------------------------------------+



#### pyspark.sql.functions.stack(*cols: ColumnOrName) → pyspark.sql.column.Column
Separates col1, …, colk into n rows. Uses column names col0, col1, etc. by default unless specified otherwise.

Parameters – cols: Column or str | the first element should be a literal int for the number of rows to be separated, and the remaining are input elements to be separated.

In [64]:
df = spark.createDataFrame([(1, 2, 3)], ["a", "b", "c"])
df.select(stack(lit(2), df.a, df.b, df.c)).show(truncate=False)

+----+----+
|col0|col1|
+----+----+
|1   |2   |
|3   |NULL|
+----+----+



In [65]:
df = spark.createDataFrame([(1, 2, 3, 4), (1, 2, 3, 4)], ["a", "b", "c", "d"])
df.select(stack(lit(2), df.a, df.b, df.c, df.d)).show(truncate=False)

+----+----+
|col0|col1|
+----+----+
|1   |2   |
|3   |4   |
|1   |2   |
|3   |4   |
+----+----+



In [66]:
df.select(stack(lit(3), df.a, df.b, df.c, df.d)).show(truncate=False)

+----+----+
|col0|col1|
+----+----+
|1   |2   |
|3   |4   |
|NULL|NULL|
|1   |2   |
|3   |4   |
|NULL|NULL|
+----+----+



In [67]:
df.select(stack(lit(1), df.a, df.b, df.c, df.d)).show(truncate=False)

+----+----+----+----+
|col0|col1|col2|col3|
+----+----+----+----+
|1   |2   |3   |4   |
|1   |2   |3   |4   |
+----+----+----+----+



In [68]:
df.select(stack(lit(4), df.a, df.b, df.c, df.d)).show(truncate=False)

+----+
|col0|
+----+
|1   |
|2   |
|3   |
|4   |
|1   |
|2   |
|3   |
|4   |
+----+



#### pyspark.sql.functions.try_aes_decrypt(input: ColumnOrName, key: ColumnOrName, mode: Optional[ColumnOrName] = None, padding: Optional[ColumnOrName] = None, aad: Optional[ColumnOrName] = None) → pyspark.sql.column.Column
This is a special version of aes_decrypt that performs the same operation, but returns a NULL value instead of raising an error if the decryption cannot be performed. Returns a decrypted value of input using AES in mode with padding. Key lengths of 16, 24 and 32 bits are supported. Supported combinations of (mode, padding) are (‘ECB’, ‘PKCS’), (‘GCM’, ‘NONE’) and (‘CBC’, ‘PKCS’). Optional additional authenticated data (AAD) is only supported for GCM. If provided for encryption, the identical AAD value must be provided for decryption. The default mode is GCM.

Parameters: 
- input: Column or str | 
The binary value to decrypt.
- key: Column or str | 
The passphrase to use to decrypt the data.
- mode: Column or str, optional |
Specifies which block cipher mode should be used to decrypt messages. Valid modes: ECB, GCM, CBC.
- padding: Column or str, optional | 
Specifies how to pad messages whose length is not a multiple of the block size. Valid values: PKCS, NONE, DEFAULT. The DEFAULT padding means PKCS for ECB, NONE for GCM and PKCS for CBC.
- aad: Column or str, optional | 
Optional additional authenticated data. Only supported for GCM mode. This can be any free-form input and must be provided for both encryption and decryption.

In [69]:
df = spark.createDataFrame([(
    "AAAAAAAAAAAAAAAAQiYi+sTLm7KD9UcZ2nlRdYDe/PX4",
    "abcdefghijklmnop12345678ABCDEFGH", "GCM", "DEFAULT",
    "This is an AAD mixed into the input",)],
    ["input", "key", "mode", "padding", "aad"]
)
df.select(try_aes_decrypt(
    unbase64(df.input), df.key, df.mode, df.padding, df.aad).alias('r')
).collect()

[Row(r=bytearray(b'Spark'))]

In [70]:
df = spark.createDataFrame([(
    "AAAAAAAAAAAAAAAAAAAAAPSd4mWyMZ5mhvjiAPQJnfg=",
    "abcdefghijklmnop12345678ABCDEFGH", "CBC", "DEFAULT",)],
    ["input", "key", "mode", "padding"]
)
df.select(try_aes_decrypt(
    unbase64(df.input), df.key, df.mode, df.padding).alias('r')
).collect()

[Row(r=bytearray(b'Spark'))]

In [71]:
df.select(try_aes_decrypt(unbase64(df.input), df.key, df.mode).alias('r')).collect()

[Row(r=bytearray(b'Spark'))]

In [72]:
df = spark.createDataFrame([(
    "83F16B2AA704794132802D248E6BFD4E380078182D1544813898AC97E709B28A94",
    "0000111122223333",)],
    ["input", "key"]
)
df.select(try_aes_decrypt(unhex(df.input), df.key).alias('r')).collect()

[Row(r=bytearray(b'Spark'))]

#### pyspark.sql.functions.typeof(col: ColumnOrName) → pyspark.sql.column.Column
Return DDL-formatted type string for the data type of the input.

In [73]:
df = spark.createDataFrame([(1,)], ["a"])
df.select(typeof(df.a).alias('r')).collect()

[Row(r='bigint')]

#### pyspark.sql.functions.user() → pyspark.sql.column.Column
Returns the current database.

In [74]:
spark.range(1).select(user()).show() 

+--------------+
|current_user()|
+--------------+
|    zsavchenko|
+--------------+



#### pyspark.sql.functions.version() → pyspark.sql.column.Column
Returns the Spark version. The string contains 2 fields, the first being a release version and the second being a git revision.

In [75]:
df = spark.range(1)
df.select(version()).show(truncate=False) 

+----------------------------------------------+
|version()                                     |
+----------------------------------------------+
|3.5.0 ce5ddad990373636e94071e7cef2f31021add07b|
+----------------------------------------------+



## [Predicate Functions](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/functions.html#predicate-functions)

Parameters:
* col1: Column or str
* col2: Column or str
* col3: Column or str

#### pyspark.sql.functions.equal_null(col1: ColumnOrName, col2: ColumnOrName) → pyspark.sql.column.Column¶
Returns same result as the EQUAL(=) operator for non-null operands, but returns true if both are null, false if one of the them is null.

In [76]:
df = spark.createDataFrame([(None, None,), (1, 9,), (1, 1,),], ["a", "b"])
df.show()
df.select(equal_null(df.a, df.b).alias('r')).collect()

+----+----+
|   a|   b|
+----+----+
|NULL|NULL|
|   1|   9|
|   1|   1|
+----+----+



[Row(r=True), Row(r=False), Row(r=True)]

#### pyspark.sql.functions.ifnull(col1: ColumnOrName, col2: ColumnOrName) → pyspark.sql.column.Column
Returns col2 if col1 is null, or col1 otherwise.

In [77]:
df = spark.createDataFrame([(None, 12,), (1, None,), (None, None,), (1, 9,),], ["e", "e2"])
df.show()
df.select(ifnull(df.e, df.e2)).show()

+----+----+
|   e|  e2|
+----+----+
|NULL|  12|
|   1|NULL|
|NULL|NULL|
|   1|   9|
+----+----+

+-------------+
|ifnull(e, e2)|
+-------------+
|           12|
|            1|
|         NULL|
|            1|
+-------------+



#### pyspark.sql.functions.isnotnull(col: ColumnOrName) → pyspark.sql.column.Column
Returns true if col is not null, or false otherwise.

In [78]:
df = spark.createDataFrame([(None,), (1,)], ["e"])
df.show()
df.select(isnotnull(df.e).alias('r')).collect()

+----+
|   e|
+----+
|NULL|
|   1|
+----+



[Row(r=False), Row(r=True)]

#### pyspark.sql.functions.nullif(col1: ColumnOrName, col2: ColumnOrName) → pyspark.sql.column.Column
Returns null if col1 equals to col2, or col1 otherwise.

In [79]:
df = spark.createDataFrame([(None, None,), (1, 9,), (1, 1,), (None, 12,), (1, None,),], ["a", "b"])
df.show()
df.select(nullif(df.a, df.b).alias('r')).collect()

+----+----+
|   a|   b|
+----+----+
|NULL|NULL|
|   1|   9|
|   1|   1|
|NULL|  12|
|   1|NULL|
+----+----+



[Row(r=None), Row(r=1), Row(r=None), Row(r=None), Row(r=1)]

#### pyspark.sql.functions.nvl(col1: ColumnOrName, col2: ColumnOrName) → pyspark.sql.column.Column
Returns col2 if col1 is null, or col1 otherwise.

In [80]:
df = spark.createDataFrame([(None, 12,), (1, None,), (None, None,), (1, 9,),], ["a", "b"])
df.show()
df.select(nvl(df.a, df.b).alias('r')).collect()

+----+----+
|   a|   b|
+----+----+
|NULL|  12|
|   1|NULL|
|NULL|NULL|
|   1|   9|
+----+----+



[Row(r=12), Row(r=1), Row(r=None), Row(r=1)]

#### pyspark.sql.functions.nvl2(col1: ColumnOrName, col2: ColumnOrName, col3: ColumnOrName) → pyspark.sql.column.Column
Returns col2 if col1 is not null, or col3 otherwise.

In [81]:
df = spark.createDataFrame([(None, 8, 6,), (1, 8, 9,)], ["a", "b", "c"])
df.select(nvl2(df.a, df.b, df.c).alias('r')).collect()

[Row(r=6), Row(r=8)]

## [Xml Functions](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/functions.html#xml-functions)

#### pyspark.sql.functions.xpath(xml: ColumnOrName, path: ColumnOrName) → pyspark.sql.column.Column
Returns a string array of values within the nodes of xml that match the XPath expression.

In [82]:
df = spark.createDataFrame(
    [('<a><b>b1</b><b>b2</b><b>b3</b><c>c1</c><c>c2</c></a>',)], ['x'])
df.select(xpath(df.x, lit('a/b/text()')).alias('r')).collect()

[Row(r=['b1', 'b2', 'b3'])]

#### pyspark.sql.functions.xpath_boolean(xml: ColumnOrName, path: ColumnOrName) → pyspark.sql.column.Column¶
Returns true if the XPath expression evaluates to true, or if a matching node is found.

In [83]:
df = spark.createDataFrame([('<a><b>1</b></a>',)], ['x'])
df.select(xpath_boolean(df.x, lit('a/b')).alias('r')).collect()

[Row(r=True)]

#### pyspark.sql.functions.xpath_double(xml: ColumnOrName, path: ColumnOrName) → pyspark.sql.column.Column
Returns a double value, the value zero if no match is found, or NaN if a match is found but the value is non-numeric.

In [84]:
df = spark.createDataFrame([('<a><b>1</b><b>2</b></a>',)], ['x'])
df.select(xpath_double(df.x, lit('sum(a/b)')).alias('r')).collect()

[Row(r=3.0)]

#### pyspark.sql.functions.xpath_int(xml: ColumnOrName, path: ColumnOrName) → pyspark.sql.column.Column
Returns an integer value, or the value zero if no match is found, or a match is found but the value is non-numeric.

In [85]:
df = spark.createDataFrame([('<a><b>1</b><b>2</b></a>',)], ['x'])
df.select(xpath_int(df.x, lit('sum(a/b)')).alias('r')).collect()

[Row(r=3)]

#### pyspark.sql.functions.xpath_long(xml: ColumnOrName, path: ColumnOrName) → pyspark.sql.column.Column
Returns a long integer value, or the value zero if no match is found, or a match is found but the value is non-numeric.

In [86]:
df = spark.createDataFrame([('<a><b>1</b><b>2</b></a>',)], ['x'])
df.select(xpath_long(df.x, lit('sum(a/b)')).alias('r')).collect()

[Row(r=3)]

#### pyspark.sql.functions.xpath_number(xml: ColumnOrName, path: ColumnOrName) → pyspark.sql.column.Column
Returns a double value, the value zero if no match is found, or NaN if a match is found but the value is non-numeric.

In [87]:
spark.createDataFrame([('<a><b>1</b><b>2</b></a>',)], ['x']).select(xpath_number('x', lit('sum(a/b)'))).show()

+-------------------------+
|xpath_number(x, sum(a/b))|
+-------------------------+
|                      3.0|
+-------------------------+



#### pyspark.sql.functions.xpath_short(xml: ColumnOrName, path: ColumnOrName) → pyspark.sql.column.Column
Returns a short integer value, or the value zero if no match is found, or a match is found but the value is non-numeric.

In [88]:
df = spark.createDataFrame([('<a><b>1</b><b>2</b></a>',)], ['x'])
df.select(xpath_short(df.x, lit('sum(a/b)')).alias('r')).collect()

[Row(r=3)]

#### pyspark.sql.functions.xpath_string(xml: ColumnOrName, path: ColumnOrName) → pyspark.sql.column.Column
Returns the text contents of the first xml node that matches the XPath expression.

In [89]:
df = spark.createDataFrame([('<a><b>b</b><c>cc</c></a>',)], ['x'])
df.select(xpath_string(df.x, lit('a/c')).alias('r')).collect()

[Row(r='cc')]